
# 🚀 Colab Setup — **CNNs-distracted-driving** (hardcoded + config-aware)

This version is **simplified and hardcoded** for your repo and URL, and it **respects your `src/ddriver/config.py`**.
- Repo name fixed to **`CNNs-distracted-driving`**
- Repo URL fixed to **`https://github.com/ClaudiaCPach/CNNs-distracted-driving`**
- Uses your `config.py` convention: when running in Colab, we **set env vars** (`DRIVE_PATH`, `DATASET_ROOT`, `OUT_ROOT`, `CKPT_ROOT`, `FAST_DATA`) so your code reads correct paths via `ddriver.config`.
- Optional `FAST_DATA` at `/content/data` for faster I/O (if you later copy data there).

> Run cells **top → bottom** the first time. Re-run **Update repo** to pull new commits after you push.


In [ ]:

# 🔧 0) (Optional) quick GPU check
!nvidia-smi || echo "No GPU detected — CPU runtime is okay for setup steps."


In [ ]:

# 🔧 1) Fixed config for your repo + Drive layout
import os

REPO_URL       = "https://github.com/ClaudiaCPach/CNNs-distracted-driving"
REPO_DIRNAME   = "CNNs-distracted-driving"   # hardcoded
BRANCH         = "main"
PROJECT_ROOT   = f"/content/{REPO_DIRNAME}"  # where the repo will live in Colab

# Your persistent Google Drive base folder (matches your project docs):
DRIVE_PATH       = "/content/drive/MyDrive/TFM"
DRIVE_DATA_ROOT  = f"{DRIVE_PATH}/data"      # contains auc.distracted.driver.dataset_v2

# Optional: a fast, ephemeral workspace inside the VM
FAST_DATA        = "/content/data"           # rsync target for faster I/O (lives on the VM SSD)

# Start with Drive as the canonical dataset root; later cells can switch to FAST_DATA
DATASET_ROOT     = DRIVE_DATA_ROOT
OUT_ROOT         = f"{DRIVE_PATH}/outputs"
CKPT_ROOT        = f"{DRIVE_PATH}/checkpoints"


In [ ]:

# 🔌 2) Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
print("✅ Drive mounted.")


In [ ]:

# 📁 3) Clone or update the repo (no name inference — all hardcoded)
import os, subprocess

def sh(cmd):
    print(f"\n$ {cmd}")
    rc = subprocess.call(cmd, shell=True, executable="/bin/bash")
    if rc != 0:
        raise RuntimeError(f"Command failed with exit code {rc}: {cmd}")

if os.path.isdir(PROJECT_ROOT):
    print(f"📁 Repo already present at {PROJECT_ROOT}. Pulling latest on branch {BRANCH}...")
    sh(f"cd {PROJECT_ROOT} && git fetch origin {BRANCH} && git checkout {BRANCH} && git pull --rebase origin {BRANCH}")
else:
    print(f"⬇️ Cloning {REPO_URL} → {PROJECT_ROOT}")
    sh(f"git clone --branch {BRANCH} {REPO_URL} {PROJECT_ROOT}")

print("PROJECT_ROOT =", PROJECT_ROOT)


In [ ]:

# 📦 4) Install the repo (editable) + requirements (uses pyproject.toml if present)
import os, subprocess

def sh(cmd):
    print(f"\n$ {cmd}")
    rc = subprocess.call(cmd, shell=True, executable="/bin/bash")
    if rc != 0:
        raise RuntimeError(f"Command failed with exit code {rc}: {cmd}")

print("🔄 Upgrading pip/setuptools/wheel...")
sh("python -m pip install --upgrade pip setuptools wheel")

has_pyproject = os.path.exists(os.path.join(PROJECT_ROOT, "pyproject.toml"))
if has_pyproject:
    print("📦 Editable install from pyproject.toml ...")
    sh(f"cd {PROJECT_ROOT} && pip install -e .")
else:
    print("⚠️ No pyproject.toml found. Skipping editable install.")

req_path = os.path.join(PROJECT_ROOT, "requirements.txt")
if os.path.exists(req_path):
    print("📝 Installing requirements.txt...")
    sh(f"pip install -r {req_path}")
else:
    print("ℹ️ No requirements.txt found — continuing.")


In [ ]:

# 🌳 5) Configure environment for your ddriver.config (Colab branch)
# Your config.py reads env vars and falls back to sensible defaults when in Colab.
import os

os.environ["DRIVE_PATH"]   = DRIVE_PATH
os.environ["DATASET_ROOT"] = DATASET_ROOT
os.environ["OUT_ROOT"]     = OUT_ROOT
os.environ["CKPT_ROOT"]    = CKPT_ROOT
os.environ["FAST_DATA"]    = FAST_DATA

# Also write a .env (harmless in Colab; helpful if code calls load_dotenv())
env_text = f"""DRIVE_PATH={DRIVE_PATH}
DATASET_ROOT={DATASET_ROOT}
OUT_ROOT={OUT_ROOT}
CKPT_ROOT={CKPT_ROOT}
FAST_DATA={FAST_DATA}
"""
with open(os.path.join(PROJECT_ROOT, ".env"), "w") as f:
    f.write(env_text)

print("✅ Environment variables set for ddriver.config")
print("\nSummary:")
for k in ["DRIVE_PATH","DATASET_ROOT","OUT_ROOT","CKPT_ROOT","FAST_DATA"]:
    print(f"{k} = {os.environ[k]}")


In [ ]:

# ✅ 8) Import smoke test (uses your package + config.py)
import sys, os
sys.path.append(PROJECT_ROOT)
sys.path.append(os.path.join(PROJECT_ROOT, "src"))  # <— lets Python find src/ddriver

try:
    import ddriver
    print("ddriver imported OK from:", ddriver.__file__)
    # Confirm config picks up Colab env:
    try:
        from ddriver import config
        print("Loaded ddriver.config successfully.")
        # Echo the resolved paths from config (they are pathlib.Path objects)
        print("config.DATASET_ROOT =", config.DATASET_ROOT)
        print("config.OUT_ROOT     =", config.OUT_ROOT)
        print("config.CKPT_ROOT    =", config.CKPT_ROOT)
        print("config.FAST_DATA    =", config.FAST_DATA)
    except Exception as e:
        print("Note: ddriver.config not imported:", e)
except Exception as e:
    print("⚠️ Import failed — check package name/setup.")
    print(e)


# 📋 9) Generate Manifest and Split CSVs

This step creates the CSV files that tell your code where all the images are and which ones go to train/val/test.

**What this does:**
- Scans all your images in the dataset folder
- Creates a big list (manifest.csv) with info about every image
- Creates three smaller lists (train.csv, val.csv, test.csv) that say which images belong where
- Saves everything to your Google Drive so it's permanent

**Why we need this:**
- Your training code needs to know which images to use
- The manifest remembers which driver each image belongs to (for VAL split)
- The split CSVs organize images into train/val/test groups


In [ ]:
# Run the manifest generator
# This is like asking a librarian to catalog all your books and create reading lists

import subprocess
import sys

# Make sure we can import ddriver
sys.path.insert(0, PROJECT_ROOT)

# Run the manifest script
# --write-split-lists means "also create train.csv, val.csv, test.csv files"
manifest_cmd = f"cd {PROJECT_ROOT} && python -m ddriver.data.manifest --write-split-lists"

print("🔨 Generating manifest and split CSVs...")
print(f"Running: {manifest_cmd}\n")

result = subprocess.run(
    manifest_cmd,
    shell=True,
    capture_output=True,
    text=True
)

# Show what happened
print(result.stdout)
if result.stderr:
    print("Warnings/Errors:")
    print(result.stderr)

if result.returncode == 0:
    print("\n✅ Manifest and split CSVs generated successfully!")
    print(f"   Manifest: {os.environ['OUT_ROOT']}/manifests/manifest.csv")
    print(f"   Train split: {os.environ['OUT_ROOT']}/splits/train.csv")
    print(f"   Val split: {os.environ['OUT_ROOT']}/splits/val.csv")
    print(f"   Test split: {os.environ['OUT_ROOT']}/splits/test.csv")
else:
    print(f"\n❌ Error generating manifest (exit code {result.returncode})")
    raise RuntimeError("Manifest generation failed")


In [ ]:
# Quick check: Did the CSVs get created?
# This is like checking that the librarian actually wrote down all the book lists

import pandas as pd
from pathlib import Path

manifest_path = Path(os.environ['OUT_ROOT']) / "manifests" / "manifest.csv"
train_path = Path(os.environ['OUT_ROOT']) / "splits" / "train.csv"
val_path = Path(os.environ['OUT_ROOT']) / "splits" / "val.csv"
test_path = Path(os.environ['OUT_ROOT']) / "splits" / "test.csv"

print("📊 Checking CSV files...\n")

for name, path in [("Manifest", manifest_path), ("Train", train_path), ("Val", val_path), ("Test", test_path)]:
    if path.exists():
        df = pd.read_csv(path)
        print(f"✅ {name}: {len(df)} rows, columns: {list(df.columns)}")
    else:
        print(f"❌ {name}: File not found at {path}")

# Show a sample from the manifest
if manifest_path.exists():
    print("\n📄 Sample from manifest (first 3 rows):")
    sample = pd.read_csv(manifest_path).head(3)
    print(sample[['path', 'class_id', 'driver_id', 'camera', 'split']].to_string())


In [ ]:
# Create a tiny balanced subset for quick testing
# Run this cell ONCE to create train_small.csv, then use it for fast experiments

import pandas as pd
from pathlib import Path
from ddriver import config

train_csv = Path(config.OUT_ROOT) / "splits" / "train.csv"
train_small_csv = Path(config.OUT_ROOT) / "splits" / "train_small.csv"

print(f"Reading {train_csv}...")
df = pd.read_csv(train_csv)

# Get 20 images per class (balanced)
small = df.groupby("class_id").head(20)

print(f"Original train.csv: {len(df)} images")
print(f"Small subset: {len(small)} images ({len(small) // 10} per class)")
print(f"\nClass distribution in small subset:")
print(small["class_id"].value_counts().sort_index())

small.to_csv(train_small_csv, index=False)
print(f"\n✅ Saved to {train_small_csv}")

### ⚡️ Tiny-train option

Set `USE_TINY_SPLIT = True` in the training cell below to replace the heavy
`train.csv` with the quick `train_small.csv` (20 images per class). Validation
and test splits stay full so you still see realistic metrics.

Run the "Create a tiny balanced subset" cell once per Drive setup before
enabling this flag.


# 🧪 10) Test dataset.py and datamod.py

Now let's make sure the code that loads images actually works!

**What we're testing:**
1. **dataset.py** - Can it load a single image and give us the right info?
2. **datamod.py** - Can it create data loaders that give us batches of images?

**Why test this:**
- If these don't work, training will fail
- Better to catch problems now than later
- We want to see that images load correctly and labels are right


## 🔍 MediaPipe Crop Quality Audit

**How to run a FAST audit (recommended):**
1. Run the "Copy crops to /content" cell 32 below (copies from Drive to fast local SSD)
2. Run the audit cell (it auto-detects the local copy and uses it)

**Two modes:**
- **Full mode**: Uses `detection_metadata_{variant}.csv` (has face/hand detection info)
- **Lite mode**: Uses `manifest_{variant}.csv` (infers fallback from crop dimensions)

**What you get:**
- Numeric stats: fallback rates, ROI area/aspect distributions
- Breakdowns by class/camera/split
- Visual grids: "worst suspects" (tiny crops, fallbacks, extreme aspects)
- Per-class sample grids

**Path conventions:** All CSVs store relative paths. At runtime, paths are resolved using `config.OUT_ROOT` or `config.FAST_DATA` depending on where the crops are.


In [ ]:
# 🔍 Run MediaPipe Crop Quality Audit
# Auto-detects whether crops are in /content (fast) or Drive, and which mode to use.

import matplotlib.pyplot as plt
from ddriver import config
from ddriver.data.mediapipe_audit import generate_audit_report, get_crop_root

VARIANT = "face_hands"  # must match the variant you extracted

# Auto-detect crop root (prefers FAST_DATA if available)
crop_root = get_crop_root(prefer_fast=True)
print(f"📁 Using crop root: {crop_root}")

# Look for metadata/manifest CSVs in the same location
metadata_csv = crop_root.parent / f"detection_metadata_{VARIANT}.csv"
manifest_csv = crop_root.parent / f"manifest_{VARIANT}.csv"

# Fall back to OUT_ROOT if not found in FAST_DATA
if not metadata_csv.exists() and not manifest_csv.exists():
    metadata_csv = config.OUT_ROOT / "mediapipe" / f"detection_metadata_{VARIANT}.csv"
    manifest_csv = config.OUT_ROOT / "mediapipe" / f"manifest_{VARIANT}.csv"

# Output directory (always on Drive for persistence)
audit_output = config.OUT_ROOT / "mediapipe" / "audit" / VARIANT

# Run the audit
if metadata_csv.exists():
    print(f"✅ Found detection metadata: {metadata_csv}")
    audit_result = generate_audit_report(
        metadata_csv=metadata_csv,
        crop_root=crop_root,
        output_dir=audit_output,
        variant=VARIANT,
        n_samples=25,
        save_figures=True,
        show_figures=True,
    )
elif manifest_csv.exists():
    print(f"⚠️ Using manifest (lite mode): {manifest_csv}")
    audit_result = generate_audit_report(
        manifest_csv=manifest_csv,
        crop_root=crop_root,
        output_dir=audit_output,
        variant=VARIANT,
        n_samples=25,
        save_figures=True,
        show_figures=True,
    )
else:
    raise FileNotFoundError(
        f"Neither metadata nor manifest found. Run extraction first.\n"
        f"  Checked: {metadata_csv}\n"
        f"  Checked: {manifest_csv}"
    )

# Store results for later cells
stats = audit_result["stats"]
breakdowns = audit_result["breakdowns"]
lite_mode = audit_result["lite_mode"]
crop_root = audit_result["crop_root"]

print(f"\n✅ Audit complete! Outputs saved to: {audit_output}")


In [ ]:
# 📊 Display Audit Summary Stats (uses results from previous cell)

print("=" * 60)
print("📊 DETECTION SUMMARY STATS")
if lite_mode:
    print("   [LITE MODE - face/hand detection info not available]")
print("=" * 60)
print(f"Total images processed: {stats['total_images']}")

# Face/hand detection (full mode only)
if not lite_mode and "face_detected_pct" in stats:
    print(f"\n🎯 Detection rates:")
    print(f"   Face detected: {stats['face_detected_pct']:.1f}%")
    print(f"   Hands: 0={stats['hands_0_pct']:.1f}%, 1={stats['hands_1_pct']:.1f}%, 2={stats['hands_2_pct']:.1f}%")

print(f"\n⚠️  Fallback to full frame: {stats['fallback_count']} ({stats['fallback_pct']:.1f}%)")
print(f"   Fallback reasons: {stats['fallback_reasons']}")

print(f"\n📐 ROI statistics:")
print(f"   Area fraction: mean={stats['roi_area_frac_mean']:.3f}, std={stats['roi_area_frac_std']:.3f}")
print(f"   Area percentiles: 5%={stats['roi_area_frac_p5']:.3f}, 25%={stats['roi_area_frac_p25']:.3f}, 50%={stats['roi_area_frac_median']:.3f}")
print(f"   Aspect ratio: mean={stats['roi_aspect_mean']:.3f}, min={stats['roi_aspect_min']:.3f}, max={stats['roi_aspect_max']:.3f}")

# Detection types (full mode only)
if not lite_mode and "detection_used_distribution" in stats:
    print(f"\n🏷️  Detection types used:")
    for dtype, count in stats['detection_used_distribution'].items():
        pct = 100 * count / stats['total_images']
        print(f"   {dtype}: {count} ({pct:.1f}%)")


In [ ]:
# 📋 Display Breakdown by Class and Camera (uses results from audit cell)
import pandas as pd

# Class breakdown
if "class_id" in breakdowns:
    print("📋 BREAKDOWN BY CLASS")
    print("-" * 100)
    print(breakdowns["class_id"].to_string(index=False))
    print()
else:
    print("⚠️ Class breakdown not available")

# Camera breakdown
if "camera" in breakdowns:
    print("📋 BREAKDOWN BY CAMERA")
    print("-" * 100)
    print(breakdowns["camera"].to_string(index=False))
    print()
else:
    print("⚠️ Camera breakdown not available")

# Split breakdown (train/val/test)
if "split" in breakdowns:
    print("📋 BREAKDOWN BY SPLIT")
    print("-" * 100)
    print(breakdowns["split"].to_string(index=False))
else:
    print("⚠️ Split breakdown not available")


In [ ]:
# 🖼️ Re-display saved grids from disk (if you need to see them again)
# Note: Grids were already shown inline when you ran the audit cell above!

from IPython.display import display, Image as IPImage
from ddriver import config

VARIANT = "face_hands"
audit_output = config.OUT_ROOT / "mediapipe" / "audit" / VARIANT

print("📁 Saved grids location:", audit_output)
print("   (These were already displayed inline during the audit)\n")

# List what's available
grids = [
    ("grid_area_small.png", "🔬 Smallest ROI crops"),
    ("grid_fallback.png", "⚠️ Fallback to full frame"),
    ("grid_aspect_extreme.png", "📐 Extreme aspect ratios"),
]
# Add full-mode only grids
if not lite_mode:
    grids.extend([
        ("grid_no_hands.png", "👤 Face detected but no hands"),
        ("grid_one_hand.png", "✋ Only one hand detected"),
    ])

for filename, title in grids:
    grid_path = audit_output / filename
    if grid_path.exists():
        print(f"✅ {title}: {grid_path.name}")
    else:
        print(f"⚠️ {title}: not found")

# Uncomment below to re-display a specific grid:
# display(IPImage(filename=str(audit_output / "grid_area_small.png"), width=900))


In [ ]:
# 🏷️ Per-Class Sample Grids info
# Note: These were already displayed inline during the audit!

from IPython.display import display, Image as IPImage
from ddriver import config

VARIANT = "face_hands"
audit_output = config.OUT_ROOT / "mediapipe" / "audit" / VARIANT

# Class labels for reference
CLASS_LABELS = {
    0: "c0 - Safe driving",
    1: "c1 - Texting (right)",
    2: "c2 - Phone (right)",
    3: "c3 - Texting (left)",
    4: "c4 - Phone (left)",
    5: "c5 - Radio",
    6: "c6 - Drinking",
    7: "c7 - Reaching behind",
    8: "c8 - Hair/makeup",
    9: "c9 - Talking to passenger",
}

print("📁 Per-class grids saved at:", audit_output)
print("   (Already displayed inline during audit)\n")

# Check what's available
for class_id in range(10):
    grid_path = audit_output / f"grid_class_{class_id}.png"
    label = CLASS_LABELS.get(class_id, f"Class {class_id}")
    if grid_path.exists():
        print(f"✅ {label}")
    else:
        print(f"⚠️ {label}: not found")

# Uncomment to re-display specific class grids:
# for class_id in [0, 1, 7]:  # Adjust class IDs as needed
#     grid_path = audit_output / f"grid_class_{class_id}.png"
#     if grid_path.exists():
#         print(f"\n🏷️ {CLASS_LABELS.get(class_id, f'Class {class_id}')}")
#         display(IPImage(filename=str(grid_path), width=900))


In [ ]:
# Test 1: Can dataset.py load a single image?
# This is like testing if a worker can fetch one book from the library

from ddriver.data.dataset import AucDriverDataset
from torchvision import transforms as T
from pathlib import Path

# Get paths from config
manifest_csv = Path(os.environ['OUT_ROOT']) / "manifests" / "manifest.csv"
val_split_csv = Path(os.environ['OUT_ROOT']) / "splits" / "val.csv"

print("🧪 Test 1: Testing AucDriverDataset (dataset.py)")
print(f"   Manifest: {manifest_csv}")
print(f"   Using Val split: {val_split_csv}\n")

try:
    # Create a simple dataset (no fancy transforms, just load the image)
    simple_transforms = T.ToTensor()  # Just convert to tensor, no augmentation
    
    val_dataset = AucDriverDataset(
        manifest_csv=manifest_csv,
        split_csv=val_split_csv,
        transforms=simple_transforms
    )
    
    print(f"✅ Dataset created! It has {len(val_dataset)} images in VAL split")
    
    # Try to load the first image
    print("\n📖 Loading first image from VAL split...")
    sample = val_dataset[0]
    
    print(f"✅ Image loaded successfully!")
    print(f"   Image shape: {sample['image'].shape} (should be [3, height, width])")
    print(f"   Label: {sample['label']} (should be 0-9)")
    print(f"   Driver ID: {sample['driver_id']} (VAL should have driver IDs)")
    print(f"   Camera: {sample['camera']} (should be 'cam1' or 'cam2')")
    print(f"   Path: {sample['path'][:80]}...")  # Show first 80 chars
    
    # Check that label is valid (0-9)
    if 0 <= sample['label'] <= 9:
        print(f"   ✅ Label is valid (0-9)")
    else:
        print(f"   ❌ Label {sample['label']} is NOT in range 0-9!")
    
    # Check that VAL has driver IDs
    if sample['driver_id'] is not None:
        print(f"   ✅ VAL split has driver ID (as expected)")
    else:
        print(f"   ⚠️  VAL split missing driver ID (might be okay if this image wasn't in your DRIVER_RANGES)")
    
    print("\n✅ Test 1 PASSED: dataset.py works!")
    
except Exception as e:
    print(f"\n❌ Test 1 FAILED: {e}")
    import traceback
    traceback.print_exc()
    raise


# 🧵 11) Full pipeline (train → predict → metrics)

Now that data loading is working, these next cells show how to:
1. Register the model you want (e.g., `resnet18` from timm)
2. Run training from the command line helper
3. Generate predictions from a checkpoint
4. Evaluate metrics and save all logs to Drive

> You can change the `RUN_TAG`, model name, epochs, etc. in the code below.


In [ ]:
# Register models you want to use (run once per runtime)
# This example uses timm's convnext_tiny.

!pip -q install timm

from ddriver.models import registry

registry.register_timm_backbone("efficientnet_b0")
print("Available models:", registry.available_models()[:10])


In [ ]:
!pip install "mediapipe==0.10.14" "protobuf<5" "opencv-python-headless<4.11"

In [ ]:
# 🧭 Generate MediaPipe ROI crops (face, hands, face+hands)
# Run once per runtime/variant. Produces new manifest/splits under OUT_ROOT/mediapipe.
!pip -q install mediapipe opencv-python-headless

import subprocess
from pathlib import Path

VARIANT = "face_hands"  # choose: face | hands | face_hands

# ===== OUTPUT LOCATION (FAST vs PERSISTENT) =====
# Option 1: Write to /content (FAST - recommended for extraction)
OUTPUT_ROOT = Path("/content/data/mediapipe")
# Option 2: Write directly to Drive (SLOW but persistent)
# OUTPUT_ROOT = Path(OUT_ROOT) / "mediapipe"

manifest_csv = Path(OUT_ROOT) / "manifests" / "manifest.csv"
splits_root = Path(OUT_ROOT) / "splits"

print(f"📁 Extraction will write to: {OUTPUT_ROOT}")
if "/content/data" in str(OUTPUT_ROOT):
    print("   ⚡ Using fast local SSD - remember to copy back to Drive when done!")
else:
    print("   💾 Writing directly to Drive (slower but persistent)")

extract_cmd = f"""
cd {PROJECT_ROOT}
python -m src.ddriver.data.mediapipe_extract \
  --manifest {manifest_csv} \
  --splits-root {splits_root} \
  --dataset-root {DATASET_ROOT} \
  --output-root {OUTPUT_ROOT} \
  --variant {VARIANT} \
  --max-side 720 \
  --model-complexity 2 \
  --min-detection-area-frac 0.05 \
  --min-area-frac 0.10 \
  --min-aspect 0.20 \
  --pad-frac 0.20 \
  --face-extra-down-frac 0.35 \
  --overwrite
"""

print("Running MediaPipe extraction for variant:", VARIANT)
print(extract_cmd)
proc = subprocess.Popen(
    extract_cmd,
    shell=True,
    text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)
if proc.stdout is None:
    raise RuntimeError("Extraction process has no stdout pipe.")
for line in proc.stdout:
    print(line, end="")
proc.wait()
if proc.returncode != 0:
    raise RuntimeError("MediaPipe extraction failed. Check logs above.")



## 🎯 YOLO-World ROI Extraction (Alternative to MediaPipe)

YOLO-World uses open-vocabulary detection to find faces and hands without custom training.
This is an alternative to the MediaPipe pipeline above.

**Advantages over MediaPipe:**
- Better detection accuracy for occluded/partial views
- Confidence scores for filtering
- Faster inference on GPU

**Choose ONE pipeline:** Either run MediaPipe extraction OR YOLO extraction, not both.
The training cell below lets you pick which pipeline's crops to use (`USE_MEDIAPIPE` vs `USE_YOLO`).

In [ ]:
# 🎯 Generate YOLO-World ROI crops (face, hands, face+hands)
# Alternative to MediaPipe - uses open-vocabulary detection.
# Run once per runtime/variant. Produces new manifest/splits under OUT_ROOT/yolo.

!pip -q install ultralytics

import subprocess
from pathlib import Path

# ===== CONFIGURATION =====
VARIANT = "face_hands"  # choose: face | hands | face_hands

# ===== OUTPUT LOCATION (FAST vs PERSISTENT) =====
# Option 1: Write to /content (FAST - recommended for extraction)
YOLO_OUTPUT_ROOT = Path("/content/data/yolo")
# Option 2: Write directly to Drive (SLOW but persistent)
# YOLO_OUTPUT_ROOT = Path(OUT_ROOT) / "yolo"

manifest_csv = Path(OUT_ROOT) / "manifests" / "manifest.csv"
splits_root = Path(OUT_ROOT) / "splits"

print(f"📁 Extraction will write to: {YOLO_OUTPUT_ROOT}")
if "/content/data" in str(YOLO_OUTPUT_ROOT):
    print("   ⚡ Using fast local SSD - remember to copy back to Drive when done!")
else:
    print("   💾 Writing directly to Drive (slower but persistent)")

# ===== TEST MODE OPTIONS (toggle these!) =====
# Option 1: Use train_small.csv for quick testing (~200 images)
TEST_MODE = True  # Set False for full extraction
SAMPLE_CSV = Path(OUT_ROOT) / "splits" / "train_small.csv"  # Small balanced subset

# Option 2: Limit to first N images (even faster for debugging)
LIMIT = None  # Set to e.g. 50 for super quick test, None for no limit

# Build command
sample_flag = f"--sample-csv {SAMPLE_CSV}" if TEST_MODE and SAMPLE_CSV.exists() else ""
limit_flag = f"--limit {LIMIT}" if LIMIT else ""

extract_cmd = f"""
cd {PROJECT_ROOT}
python -m src.ddriver.data.yolo_extract \
  --manifest {manifest_csv} \
  --splits-root {splits_root} \
  --dataset-root {DATASET_ROOT} \
  --output-root {YOLO_OUTPUT_ROOT} \
  --variant {VARIANT} \
  --model-size m \
  --confidence 0.15 \
  --min-detection-area-frac 0.03 \
  --min-area-frac 0.08 \
  --min-aspect 0.20 \
  --pad-frac 0.20 \
  {sample_flag} \
  {limit_flag} \
  --overwrite
"""

if TEST_MODE:
    print("⚡ TEST MODE: Using small sample for quick testing")
    print(f"   Sample CSV: {SAMPLE_CSV}")
    if LIMIT:
        print(f"   Limit: {LIMIT} images")
else:
    print("🪵 FULL MODE: Processing all images")

print(f"\nRunning YOLO-World extraction for variant: {VARIANT}")
print(extract_cmd)
proc = subprocess.Popen(
    extract_cmd,
    shell=True,
    text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)
if proc.stdout is None:
    raise RuntimeError("Extraction process has no stdout pipe.")
for line in proc.stdout:
    print(line, end="")
proc.wait()
if proc.returncode != 0:
    raise RuntimeError("YOLO extraction failed. Check logs above.")


## 🚚 Copy YOLO Crops (Fast ↔ Drive)

**Two modes:**
1. **After extraction to /content** → Copy TO Drive for persistence
2. **If extracted to Drive** → Copy FROM Drive to /content for faster training

Use this if training/audit from Drive is slow. It copies the generated YOLO crops/CSVs into `/content/data/yolo/<variant>` and updates paths for the current runtime.


In [ ]:
# 🚚 Copy YOLO crops (Fast ↔ Drive)
import os, shutil
from pathlib import Path

# Set your variant to the one you extracted already
YOLO_VARIANT = "face_hands"  # face | hands | face_hands

# Define both locations
LOCAL_ROOT = Path("/content/data/yolo")
DRIVE_ROOT = Path(OUT_ROOT) / "yolo"

LOCAL_VARIANT_DIR = LOCAL_ROOT / YOLO_VARIANT
DRIVE_VARIANT_DIR = DRIVE_ROOT / YOLO_VARIANT

# ===== AUTO-DETECT COPY DIRECTION =====
local_exists = LOCAL_VARIANT_DIR.exists() and any(LOCAL_VARIANT_DIR.glob("**/*.jpg"))
drive_exists = DRIVE_VARIANT_DIR.exists() and any(DRIVE_VARIANT_DIR.glob("**/*.jpg"))

if local_exists and not drive_exists:
    COPY_MODE = "TO_DRIVE"
    SRC_ROOT = LOCAL_ROOT
    SRC_VARIANT_DIR = LOCAL_VARIANT_DIR
    DST_ROOT = DRIVE_VARIANT_DIR
    print("📦 Mode: Copy FROM /content TO Drive (for persistence)")
elif drive_exists and not local_exists:
    COPY_MODE = "TO_LOCAL"
    SRC_ROOT = DRIVE_ROOT
    SRC_VARIANT_DIR = DRIVE_VARIANT_DIR
    DST_ROOT = LOCAL_VARIANT_DIR
    print("📦 Mode: Copy FROM Drive TO /content (for faster training)")
elif local_exists and drive_exists:
    print("⚠️  Crops exist in BOTH locations!")
    COPY_MODE = "TO_DRIVE"
    SRC_ROOT = LOCAL_ROOT
    SRC_VARIANT_DIR = LOCAL_VARIANT_DIR
    DST_ROOT = DRIVE_VARIANT_DIR
    print("   Defaulting to: Copy TO Drive for persistence")
else:
    raise FileNotFoundError(
        f"No crops found in either location!\n"
        f"   /content: {LOCAL_VARIANT_DIR}\n"
        f"   Drive:    {DRIVE_VARIANT_DIR}\n"
        f"Run the YOLO extraction cell first."
    )

print(f"Copying YOLO crops from {SRC_VARIANT_DIR} -> {DST_ROOT}")
DST_ROOT.mkdir(parents=True, exist_ok=True)

# Copy images directory tree
file_count = 0
for src_dir, _, files in os.walk(SRC_VARIANT_DIR):
    rel_dir = Path(src_dir).relative_to(SRC_VARIANT_DIR)
    dst_dir = DST_ROOT / rel_dir
    dst_dir.mkdir(parents=True, exist_ok=True)
    for fname in files:
        src_path = Path(src_dir) / fname
        dst_path = dst_dir / fname
        if not dst_path.exists():
            shutil.copy2(src_path, dst_path)
            file_count += 1
print(f"   Copied {file_count} image files")

# Copy manifest/split CSVs that live one level above the variant folder
csv_names = [
    f"manifest_{YOLO_VARIANT}.csv",
    f"train_{YOLO_VARIANT}.csv",
    f"val_{YOLO_VARIANT}.csv",
    f"test_{YOLO_VARIANT}.csv",
    f"detection_metadata_{YOLO_VARIANT}.csv",  # for auditing
]

# Determine CSV destination
dst_csv_root = DST_ROOT.parent if COPY_MODE == "TO_LOCAL" else DRIVE_ROOT

for fname in csv_names:
    src_csv = SRC_ROOT / fname
    if not src_csv.exists():
        print(f"   ⚠️ Skipping {fname} (not found)")
        continue
    dst_csv = dst_csv_root / fname
    shutil.copy2(src_csv, dst_csv)
    print(f"   ✅ Copied {fname}")

# ===== SET ENVIRONMENT VARIABLES =====
# Always point to /content for fast training
os.environ["YOLO_ROOT_LOCAL"] = str(LOCAL_ROOT)
os.environ["DATASET_ROOT"] = str(LOCAL_ROOT)
import importlib
from ddriver import config as _cfg
importlib.reload(_cfg)

print(f"\n✅ Copy complete!")
if COPY_MODE == "TO_DRIVE":
    print(f"   💾 Crops persisted to Drive: {DRIVE_ROOT}")
    print(f"   ⚡ Training will use fast local copy: {LOCAL_ROOT}")
else:
    print(f"   ⚡ Crops copied to fast local SSD: {LOCAL_ROOT}")
    
print(f"\n   YOLO_ROOT_LOCAL = {os.environ['YOLO_ROOT_LOCAL']}")
print(f"   DATASET_ROOT    = {os.environ['DATASET_ROOT']} (images loaded from here)")
print(f"\n   Set USE_YOLO=True in training cell.")


## 🔍 YOLO Crop Quality Audit

Quick stats and visual inspection of YOLO-World crops. Uses `detection_metadata_{variant}.csv`.


In [ ]:
# 🔍 YOLO Crop Quality Audit
import pandas as pd
import numpy as np
from pathlib import Path
from ddriver import config

VARIANT = "face_hands"  # must match the variant you extracted

# Auto-detect: prefer local copy if available
yolo_root_local = Path(os.environ.get("YOLO_ROOT_LOCAL", ""))
if yolo_root_local.exists():
    yolo_root = yolo_root_local
else:
    yolo_root = Path(OUT_ROOT) / "yolo"

metadata_csv = yolo_root / f"detection_metadata_{VARIANT}.csv"
if not metadata_csv.exists():
    raise FileNotFoundError(f"Detection metadata not found: {metadata_csv}\nRun YOLO extraction first.")

print(f"📁 Loading metadata from: {metadata_csv}")
df = pd.read_csv(metadata_csv)

# Summary stats
n_total = len(df)
n_fallback = df["fallback_to_full"].sum()
n_face = (df["face_count"] > 0).sum()
n_hands = (df["hand_count"] > 0).sum()
n_face_and_hands = ((df["face_count"] > 0) & (df["hand_count"] > 0)).sum()
avg_face_conf = df.loc[df["face_confidence"] > 0, "face_confidence"].mean()
avg_hand_conf = df.loc[df["hand_confidence"] > 0, "hand_confidence"].mean()

print("=" * 60)
print("📊 YOLO DETECTION SUMMARY")
print("=" * 60)
print(f"Total images: {n_total}")
print(f"\n🎯 Detection rates:")
print(f"   Face detected: {n_face} ({100*n_face/n_total:.1f}%)")
print(f"   Hands detected: {n_hands} ({100*n_hands/n_total:.1f}%)")
print(f"   Both face+hands: {n_face_and_hands} ({100*n_face_and_hands/n_total:.1f}%)")
print(f"\n📈 Confidence scores:")
print(f"   Avg face confidence: {avg_face_conf:.3f}" if not np.isnan(avg_face_conf) else "   Avg face confidence: N/A")
print(f"   Avg hand confidence: {avg_hand_conf:.3f}" if not np.isnan(avg_hand_conf) else "   Avg hand confidence: N/A")
print(f"\n⚠️  Fallback to full frame: {n_fallback} ({100*n_fallback/n_total:.1f}%)")

# Fallback reason breakdown
fallback_df = df[df["fallback_to_full"]]
if len(fallback_df) > 0:
    print(f"   Fallback reasons:")
    for reason, count in fallback_df["fallback_reason"].value_counts().items():
        print(f"      - {reason}: {count} ({100*count/n_total:.1f}%)")

# ROI stats (for non-fallback images)
non_fallback = df[~df["fallback_to_full"]]
if len(non_fallback) > 0:
    print(f"\n📐 ROI statistics (non-fallback only, n={len(non_fallback)}):")
    print(f"   Raw detection area: mean={non_fallback['raw_detection_area_frac'].mean():.3f}, "
          f"std={non_fallback['raw_detection_area_frac'].std():.3f}")
    print(f"   Final ROI area: mean={non_fallback['roi_area_frac'].mean():.3f}, "
          f"std={non_fallback['roi_area_frac'].std():.3f}")
    print(f"   Aspect ratio: mean={non_fallback['roi_aspect'].mean():.3f}, "
          f"min={non_fallback['roi_aspect'].min():.3f}, max={non_fallback['roi_aspect'].max():.3f}")


In [ ]:
# 📋 YOLO Breakdown by Camera and Class
print("\n📋 BREAKDOWN BY CAMERA")
print("-" * 80)
camera_stats = df.groupby("camera").agg({
    "fallback_to_full": ["sum", "mean"],
    "roi_area_frac": "mean",
    "face_count": lambda x: (x > 0).mean(),
    "hand_count": lambda x: (x > 0).mean(),
}).round(3)
camera_stats.columns = ["fallback_count", "fallback_pct", "mean_roi_area", "face_rate", "hand_rate"]
camera_stats["fallback_pct"] = (camera_stats["fallback_pct"] * 100).round(1)
camera_stats["face_rate"] = (camera_stats["face_rate"] * 100).round(1)
camera_stats["hand_rate"] = (camera_stats["hand_rate"] * 100).round(1)
print(camera_stats.to_string())

print("\n📋 BREAKDOWN BY CLASS")
print("-" * 80)
class_stats = df.groupby("class_id").agg({
    "fallback_to_full": ["sum", "mean"],
    "roi_area_frac": "mean",
    "face_count": lambda x: (x > 0).mean(),
    "hand_count": lambda x: (x > 0).mean(),
}).round(3)
class_stats.columns = ["fallback_count", "fallback_pct", "mean_roi_area", "face_rate", "hand_rate"]
class_stats["fallback_pct"] = (class_stats["fallback_pct"] * 100).round(1)
class_stats["face_rate"] = (class_stats["face_rate"] * 100).round(1)
class_stats["hand_rate"] = (class_stats["hand_rate"] * 100).round(1)
print(class_stats.to_string())


In [ ]:
# 🖼️ Visual Sample Grid - YOLO Crops
import matplotlib.pyplot as plt
import cv2

def show_sample_grid(df_subset, title, crop_root, n_samples=12, n_cols=4):
    """Display a grid of sample crops from a DataFrame subset."""
    samples = df_subset.sample(n=min(n_samples, len(df_subset)), random_state=42)
    n_rows = (len(samples) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 4*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for ax, (_, row) in zip(axes, samples.iterrows()):
        crop_path = crop_root / row["cropped_path"]
        if crop_path.exists():
            img = cv2.imread(str(crop_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            ax.imshow(img)
            label = f"c{int(row['class_id'])} | area={row['roi_area_frac']:.2f}"
            if row["fallback_to_full"]:
                label += " [FALLBACK]"
            ax.set_title(label, fontsize=9)
        else:
            ax.text(0.5, 0.5, "Not found", ha="center", va="center")
        ax.axis("off")
    
    # Hide empty subplots
    for ax in axes[len(samples):]:
        ax.axis("off")
    
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

# Show samples: fallbacks, smallest ROIs, by class
crop_root = yolo_root  # Use the root determined in the audit cell

# Fallback samples
fallback_samples = df[df["fallback_to_full"]]
if len(fallback_samples) > 0:
    show_sample_grid(fallback_samples, "⚠️ Fallback to Full Frame Samples", crop_root)

# Smallest ROI samples (non-fallback)
smallest = non_fallback.nsmallest(12, "roi_area_frac")
if len(smallest) > 0:
    show_sample_grid(smallest, "🔬 Smallest ROI Crops (non-fallback)", crop_root)

# Per-class samples (one random per class)
print("\n🏷️ Sample crop per class:")
for class_id in sorted(df["class_id"].unique()):
    class_df = df[df["class_id"] == class_id]
    if len(class_df) > 0:
        show_sample_grid(class_df, f"Class {int(class_id)} Samples", crop_root, n_samples=8, n_cols=4)


## 🔀 Hybrid ROI Extraction (RetinaFace + MediaPipe Hands)

**Best accuracy option!** Uses specialized models:
- **RetinaFace**: State-of-the-art face detection (handles occlusion, angles)
- **MediaPipe Hands**: Google's dedicated hand model (much better than Holistic)

This typically gives the lowest fallback rate and best hand detection.


In [ ]:
# 🔀 Generate Hybrid ROI crops (InsightFace + MediaPipe Hands)
# Best accuracy option - uses specialized models for face and hands.
# InsightFace uses ONNX (no TensorFlow dependency!)
!pip -q install insightface onnxruntime mediapipe

import subprocess
from pathlib import Path

# ===== CONFIGURATION =====
VARIANT = "face_hands"  # choose: face | hands | face_hands

# ===== OUTPUT LOCATION (FAST vs PERSISTENT) =====
# Option 1: Write to /content (FAST - recommended for extraction)
HYBRID_OUTPUT_ROOT = Path("/content/data/hybrid")
# Option 2: Write directly to Drive (SLOW but persistent)
# HYBRID_OUTPUT_ROOT = Path(OUT_ROOT) / "hybrid"

manifest_csv = Path(OUT_ROOT) / "manifests" / "manifest.csv"
splits_root = Path(OUT_ROOT) / "splits"

print(f"📁 Extraction will write to: {HYBRID_OUTPUT_ROOT}")
if "/content/data" in str(HYBRID_OUTPUT_ROOT):
    print("   ⚡ Using fast local SSD - remember to copy back to Drive when done!")
else:
    print("   💾 Writing directly to Drive (slower but persistent)")

# ===== AUTO-DETECT LOCAL vs DRIVE IMAGES =====
# If you ran the "copy + compress dataset" cells (46/47), images will be in /content/data
# We auto-detect and use local images for faster I/O, falling back to Drive if not available
LOCAL_DATASET_ROOT = Path("/content/data/auc.distracted.driver.dataset_v2")
DRIVE_DATASET_ROOT = Path(DATASET_ROOT)

if LOCAL_DATASET_ROOT.exists() and any(LOCAL_DATASET_ROOT.iterdir()):
    EFFECTIVE_DATASET_ROOT = LOCAL_DATASET_ROOT
    print(f"🚀 FAST MODE: Using local images from {LOCAL_DATASET_ROOT}")
else:
    EFFECTIVE_DATASET_ROOT = DRIVE_DATASET_ROOT
    print(f"📁 Using images from Drive: {DRIVE_DATASET_ROOT}")
    print("   💡 Tip: Run cells 46/47 first to copy images to /content for faster extraction!")

# ===== TEST MODE OPTIONS (toggle these!) =====
# Option 1: Use train_small.csv for quick testing (~200 images)
TEST_MODE = True  # Set False for full extraction
SAMPLE_CSV = Path(OUT_ROOT) / "splits" / "train_small.csv"  # Small balanced subset

# Option 2: Limit to first N images (even faster for debugging)
LIMIT = None  # Set to e.g. 50 for super quick test, None for no limit

# Build command
sample_flag = f"--sample-csv {SAMPLE_CSV}" if TEST_MODE and SAMPLE_CSV.exists() else ""
limit_flag = f"--limit {LIMIT}" if LIMIT else ""

extract_cmd = f"""
cd {PROJECT_ROOT}
python -m src.ddriver.data.hybrid_extract \
  --manifest {manifest_csv} \
  --splits-root {splits_root} \
  --dataset-root {EFFECTIVE_DATASET_ROOT} \
  --output-root {HYBRID_OUTPUT_ROOT} \
  --variant {VARIANT} \
  --min-detection-area-frac 0.005 \
  --min-area-frac 0.01 \
  --min-aspect 0.08 \
  --pad-frac 0.35 \
  --max-area-frac 0.40 \
  {sample_flag} \
  {limit_flag} \
  --overwrite
"""

if TEST_MODE:
    print("⚡ TEST MODE: Using small sample for quick testing")
    print(f"   Sample CSV: {SAMPLE_CSV}")
    if LIMIT:
        print(f"   Limit: {LIMIT} images")
else:
    print("🪵 FULL MODE: Processing all images")

print(f"\nRunning Hybrid extraction (InsightFace + MediaPipe Hands) for variant: {VARIANT}")
print(extract_cmd)
proc = subprocess.Popen(
    extract_cmd,
    shell=True,
    text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)
if proc.stdout is None:
    raise RuntimeError("Extraction process has no stdout pipe.")
for line in proc.stdout:
    print(line, end="")
proc.wait()
if proc.returncode != 0:
    raise RuntimeError("Hybrid extraction failed. Check logs above.")


In [ ]:
# 🔁 Regenerate Hybrid CSVs (manifest + splits)
# Use this after extraction to rebuild CSVs that point to crop paths.
from pathlib import Path
import pandas as pd

# ===== CONFIGURATION =====
VARIANT = VARIANT if 'VARIANT' in globals() else 'face_hands'
HYBRID_OUTPUT_ROOT = HYBRID_OUTPUT_ROOT if 'HYBRID_OUTPUT_ROOT' in globals() else Path('/content/data/hybrid')
HYBRID_OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)
OUT_ROOT = Path(OUT_ROOT) if 'OUT_ROOT' in globals() else Path('/content/drive/MyDrive')

manifest_csv = Path(OUT_ROOT) / 'manifests' / 'manifest.csv'
splits_root = Path(OUT_ROOT) / 'splits'
crop_root = Path(HYBRID_OUTPUT_ROOT) / VARIANT
meta_csv = Path(HYBRID_OUTPUT_ROOT) / f'detection_metadata_{VARIANT}.csv'

def _extract_class(path_str):
    for part in Path(path_str).parts:
        if len(part) == 2 and part.startswith('c') and part[1].isdigit():
            return part
    return None

def _extract_camera(path_str):
    for part in Path(path_str).parts:
        if part.lower().startswith('camera'):
            return part
    return None

def _extract_filename(path_str):
    return Path(path_str).name

def _coerce_class_id(value):
    if pd.isna(value):
        return None
    value_str = str(value)
    if len(value_str) == 2 and value_str.startswith('c') and value_str[1].isdigit():
        return value_str
    if value_str.isdigit():
        return f'c{int(value_str)}'
    return None

def _normalize_camera(cam):
    """Normalize camera values: 'Camera 1' -> 'cam1', 'cam2' -> 'cam2', etc."""
    if cam is None or pd.isna(cam):
        return None
    cam_str = str(cam).lower().replace(' ', '')
    if cam_str in ['camera1', 'cam1']:
        return 'cam1'
    if cam_str in ['camera2', 'cam2']:
        return 'cam2'
    return cam_str

print(f'📂 Loading original manifest: {manifest_csv}')
orig_df = pd.read_csv(manifest_csv)
orig_df = orig_df.rename(columns={'path': 'original_path'})
orig_df['_filename'] = orig_df['original_path'].astype(str).map(_extract_filename)
orig_df['_class'] = orig_df['original_path'].astype(str).map(_extract_class)
orig_df['_camera'] = orig_df['original_path'].astype(str).map(_extract_camera).map(_normalize_camera)

print(f'🔍 Scanning crops: {crop_root}')
crop_paths = list(crop_root.rglob('*.jpg'))
crop_df = pd.DataFrame({'crop_path': [str(p) for p in crop_paths]})
crop_df['path'] = crop_df['crop_path'].map(lambda p: str(Path(p).relative_to(HYBRID_OUTPUT_ROOT)))
crop_df['_filename'] = crop_df['crop_path'].map(_extract_filename)
crop_df['_class'] = crop_df['crop_path'].map(_extract_class)
crop_df['_camera'] = crop_df['crop_path'].map(_extract_camera).map(_normalize_camera)

fallback_paths = set()
if meta_csv.exists():
    meta_df = pd.read_csv(meta_csv)
    meta_df = meta_df[meta_df['cropped_path'].astype(str).str.len() > 0]
    meta_df['path'] = meta_df['cropped_path'].astype(str)
    meta_df['_class'] = meta_df['class_id'].map(_coerce_class_id)
    meta_df['_camera'] = meta_df['camera'].map(_normalize_camera)
    crop_df = crop_df.merge(
        meta_df[['path', '_class', '_camera', 'fallback_to_full']],
        on='path',
        how='left',
        suffixes=('', '_meta'),
    )
    crop_df['_class'] = crop_df['_class'].fillna(crop_df['_class_meta'])
    crop_df['_camera'] = crop_df['_camera'].fillna(crop_df['_camera_meta'])
    crop_df = crop_df.drop(columns=['_class_meta', '_camera_meta'], errors='ignore')
    fallback_paths = set(meta_df.loc[meta_df['fallback_to_full'] == True, 'path'].dropna().astype(str))
    print(f'🚫 Excluding {len(fallback_paths)} fallback crops from splits')

crop_df_all = crop_df.copy()
crop_df = crop_df[~crop_df['path'].isin(fallback_paths)]

merged = crop_df_all.merge(orig_df, on=['_filename', '_class', '_camera'], how='left')
manifest_out = merged.drop(columns=['crop_path', '_filename', '_class', '_camera'], errors='ignore')
manifest_out_path = Path(HYBRID_OUTPUT_ROOT) / f'manifest_{VARIANT}.csv'
manifest_out.to_csv(manifest_out_path, index=False)
print(f'✅ Wrote manifest: {manifest_out_path}')

for split_name in ['train', 'val', 'test']:
    split_path = splits_root / f'{split_name}.csv'
    split_df = pd.read_csv(split_path)
    split_df['path'] = split_df['path'].astype(str)
    split_df['_filename'] = split_df['path'].map(_extract_filename)
    split_df['_class'] = split_df['path'].map(_extract_class)
    split_df['_camera'] = split_df['path'].map(_extract_camera).map(_normalize_camera)

    split_merged = split_df.merge(
        crop_df,
        on=['_filename', '_class', '_camera'],
        how='inner',
    )
    
    # FIXED: Explicitly set path to crop path (path_y) and rename path_x to original_path
    split_merged['original_path'] = split_merged['path_x']
    split_merged['path'] = split_merged['path_y']
    
    # Drop temporary columns
    cols_to_drop = ['path_x', 'path_y', '_filename', '_class', '_camera', 'crop_path', 'fallback_to_full']
    split_merged = split_merged.drop(columns=[c for c in cols_to_drop if c in split_merged.columns])

    out_split = Path(HYBRID_OUTPUT_ROOT) / f'{split_name}_{VARIANT}.csv'
    split_merged.to_csv(out_split, index=False)
    print(f'✅ Wrote split: {out_split} ({len(split_merged)} rows)')
    
    # Verify the path column is correct
    print(f"   path sample: {split_merged['path'].iloc[0]}")

## 🚚 Copy Hybrid Crops (Fast ↔ Drive)

**Two modes:**
1. **After extraction to /content** → Copy TO Drive for persistence
2. **If extracted to Drive** → Copy FROM Drive to /content for faster training

Run this cell after extraction is complete.

Use this if training/audit from Drive is slow. Copies crops to local SSD.


In [ ]:
# 🚚 Copy Hybrid crops (Fast ↔ Drive)
import os, shutil
from pathlib import Path

# Set your variant to the one you extracted already
HYBRID_VARIANT = "face_hands"  # face | hands | face_hands

# Define both locations
LOCAL_ROOT = Path("/content/data/hybrid")
DRIVE_ROOT = Path(OUT_ROOT) / "hybrid"

LOCAL_VARIANT_DIR = LOCAL_ROOT / HYBRID_VARIANT
DRIVE_VARIANT_DIR = DRIVE_ROOT / HYBRID_VARIANT

# ===== AUTO-DETECT COPY DIRECTION =====
local_exists = LOCAL_VARIANT_DIR.exists() and any(LOCAL_VARIANT_DIR.glob("**/*.jpg"))
drive_exists = DRIVE_VARIANT_DIR.exists() and any(DRIVE_VARIANT_DIR.glob("**/*.jpg"))

if local_exists and not drive_exists:
    # Extracted to /content, need to copy TO Drive for persistence
    COPY_MODE = "TO_DRIVE"
    SRC_ROOT = LOCAL_ROOT
    SRC_VARIANT_DIR = LOCAL_VARIANT_DIR
    DST_ROOT = DRIVE_VARIANT_DIR
    print("📦 Mode: Copy FROM /content TO Drive (for persistence)")
elif drive_exists and not local_exists:
    # Extracted to Drive, copy TO /content for faster training
    COPY_MODE = "TO_LOCAL"
    SRC_ROOT = DRIVE_ROOT
    SRC_VARIANT_DIR = DRIVE_VARIANT_DIR
    DST_ROOT = LOCAL_VARIANT_DIR
    print("📦 Mode: Copy FROM Drive TO /content (for faster training)")
elif local_exists and drive_exists:
    # Both exist - ask user which direction
    print("⚠️  Crops exist in BOTH locations!")
    print(f"   /content: {LOCAL_VARIANT_DIR}")
    print(f"   Drive:    {DRIVE_VARIANT_DIR}")
    COPY_MODE = "TO_DRIVE"  # Default: persist to Drive
    SRC_ROOT = LOCAL_ROOT
    SRC_VARIANT_DIR = LOCAL_VARIANT_DIR
    DST_ROOT = DRIVE_VARIANT_DIR
    print("   Defaulting to: Copy TO Drive for persistence")
    print("   (Change COPY_MODE = 'TO_LOCAL' if you want the other direction)")
else:
    raise FileNotFoundError(
        f"No crops found in either location!\n"
        f"   /content: {LOCAL_VARIANT_DIR}\n"
        f"   Drive:    {DRIVE_VARIANT_DIR}\n"
        f"Run the Hybrid extraction cell first."
    )

print(f"Copying Hybrid crops from {SRC_VARIANT_DIR} -> {DST_ROOT}")

# Delete old destination directory to remove files that were skipped in new extraction
if COPY_MODE == "TO_DRIVE" and DST_ROOT.exists():
    print(f"   🗑️  Deleting old crops in {DST_ROOT} to ensure clean copy...")
    shutil.rmtree(DST_ROOT)

DST_ROOT.mkdir(parents=True, exist_ok=True)

# Copy images directory tree
file_count = 0
for src_dir, _, files in os.walk(SRC_VARIANT_DIR):
    rel_dir = Path(src_dir).relative_to(SRC_VARIANT_DIR)
    dst_dir = DST_ROOT / rel_dir
    dst_dir.mkdir(parents=True, exist_ok=True)
    for fname in files:
        src_path = Path(src_dir) / fname
        dst_path = dst_dir / fname
        if not dst_path.exists():
            shutil.copy2(src_path, dst_path)
            file_count += 1
print(f"   Copied {file_count} image files")

# Copy manifest/split CSVs that live one level above the variant folder
csv_names = [
    f"manifest_{HYBRID_VARIANT}.csv",
    f"train_{HYBRID_VARIANT}.csv",
    f"val_{HYBRID_VARIANT}.csv",
    f"test_{HYBRID_VARIANT}.csv",
    f"detection_metadata_{HYBRID_VARIANT}.csv",  # for auditing
]

# Determine CSV destination (parent of variant dir)
dst_csv_root = DST_ROOT.parent if COPY_MODE == "TO_LOCAL" else DRIVE_ROOT

for fname in csv_names:
    src_csv = SRC_ROOT / fname
    if not src_csv.exists():
        print(f"   ⚠️ Skipping {fname} (not found)")
        continue
    dst_csv = dst_csv_root / fname
    shutil.copy2(src_csv, dst_csv)
    print(f"   ✅ Copied {fname}")

# ===== SET ENVIRONMENT VARIABLES =====
# Always point HYBRID_ROOT_LOCAL to /content for fast training
os.environ["HYBRID_ROOT_LOCAL"] = str(LOCAL_ROOT)

# CRITICAL: Also update DATASET_ROOT so the Dataset class loads images from local copy!
os.environ["DATASET_ROOT"] = str(LOCAL_ROOT)
import importlib
from ddriver import config as _cfg
importlib.reload(_cfg)

print(f"\n✅ Copy complete!")
if COPY_MODE == "TO_DRIVE":
    print(f"   💾 Crops persisted to Drive: {DRIVE_ROOT}")
    print(f"   ⚡ Training will use fast local copy: {LOCAL_ROOT}")
else:
    print(f"   ⚡ Crops copied to fast local SSD: {LOCAL_ROOT}")
    
print(f"\n   HYBRID_ROOT_LOCAL = {os.environ['HYBRID_ROOT_LOCAL']}")
print(f"   DATASET_ROOT      = {os.environ['DATASET_ROOT']} (images loaded from here)")
print(f"\n   Set USE_HYBRID=True in training cell.")


## 🔍 Hybrid Crop Quality Audit

Quick stats and visual inspection of Hybrid crops (InsightFace + MediaPipe Hands).


In [ ]:
# 🔍 Hybrid Crop Quality Audit
import pandas as pd
import numpy as np
from pathlib import Path

VARIANT = "face_hands"  # must match the variant you extracted

# Auto-detect: prefer local copy if available
hybrid_root_local = Path(os.environ.get("HYBRID_ROOT_LOCAL", ""))
if hybrid_root_local.exists():
    hybrid_root = hybrid_root_local
else:
    hybrid_root = Path(OUT_ROOT) / "hybrid"

metadata_csv = hybrid_root / f"detection_metadata_{VARIANT}.csv"
if not metadata_csv.exists():
    raise FileNotFoundError(f"Detection metadata not found: {metadata_csv}\nRun Hybrid extraction first.")

print(f"📁 Loading metadata from: {metadata_csv}")
df = pd.read_csv(metadata_csv)

# Summary stats
n_total = len(df)
n_fallback = df["fallback_to_full"].sum()
# Handle new 'skipped' column (may not exist in old metadata CSVs)
n_skipped = df["skipped"].sum() if "skipped" in df.columns else 0
n_saved = n_total - n_skipped
n_face = (df["face_count"] > 0).sum()
n_left_hand = df["left_hand_detected"].sum()
n_right_hand = df["right_hand_detected"].sum()
n_both_hands = ((df["left_hand_detected"]) & (df["right_hand_detected"])).sum()
n_any_hands = ((df["left_hand_detected"]) | (df["right_hand_detected"])).sum()
n_face_and_hands = ((df["face_count"] > 0) & (n_any_hands > 0)).sum()
avg_face_conf = df.loc[df["face_confidence"] > 0, "face_confidence"].mean()
avg_left_conf = df.loc[df["left_hand_confidence"] > 0, "left_hand_confidence"].mean()
avg_right_conf = df.loc[df["right_hand_confidence"] > 0, "right_hand_confidence"].mean()

print("=" * 60)
print("📊 HYBRID DETECTION SUMMARY (RetinaFace + MediaPipe Hands)")
print("=" * 60)
print(f"Total images processed: {n_total}")
if n_skipped > 0:
    print(f"   Images SAVED: {n_saved} ({100*n_saved/n_total:.1f}%)")
    print(f"   Images SKIPPED: {n_skipped} ({100*n_skipped/n_total:.1f}%)")
print(f"\n🎯 Detection rates:")
print(f"   Face detected (RetinaFace): {n_face} ({100*n_face/n_total:.1f}%)")
print(f"   Left hand (MediaPipe): {n_left_hand} ({100*n_left_hand/n_total:.1f}%)")
print(f"   Right hand (MediaPipe): {n_right_hand} ({100*n_right_hand/n_total:.1f}%)")
print(f"   Both hands: {n_both_hands} ({100*n_both_hands/n_total:.1f}%)")
print(f"   Any hand: {n_any_hands} ({100*n_any_hands/n_total:.1f}%)")
print(f"\n📈 Confidence scores:")
print(f"   Avg face confidence: {avg_face_conf:.3f}" if not np.isnan(avg_face_conf) else "   Avg face confidence: N/A")
print(f"   Avg left hand confidence: {avg_left_conf:.3f}" if not np.isnan(avg_left_conf) else "   Avg left hand confidence: N/A")
print(f"   Avg right hand confidence: {avg_right_conf:.3f}" if not np.isnan(avg_right_conf) else "   Avg right hand confidence: N/A")
print(f"\n⚠️  Fallback to full frame: {n_fallback} ({100*n_fallback/n_total:.1f}%)")

# Fallback reason breakdown
fallback_df = df[df["fallback_to_full"]]
if len(fallback_df) > 0:
    print(f"   Fallback reasons:")
    for reason, count in fallback_df["fallback_reason"].value_counts().items():
        print(f"      - {reason}: {count} ({100*count/n_total:.1f}%)")

# ROI stats (for non-fallback images)
non_fallback = df[~df["fallback_to_full"]]
if len(non_fallback) > 0:
    print(f"\n📐 ROI statistics (non-fallback only, n={len(non_fallback)}):")
    print(f"   Raw detection area: mean={non_fallback['raw_detection_area_frac'].mean():.3f}, "
          f"std={non_fallback['raw_detection_area_frac'].std():.3f}")
    print(f"   Final ROI area: mean={non_fallback['roi_area_frac'].mean():.3f}, "
          f"std={non_fallback['roi_area_frac'].std():.3f}")


In [ ]:
# 📋 Hybrid Breakdown by Camera and Class
print("\n📋 BREAKDOWN BY CAMERA")
print("-" * 80)
camera_stats = df.groupby("camera").agg({
    "fallback_to_full": ["sum", "mean"],
    "roi_area_frac": "mean",
    "face_count": lambda x: (x > 0).mean(),
    "left_hand_detected": "mean",
    "right_hand_detected": "mean",
}).round(3)
camera_stats.columns = ["fallback_count", "fallback_pct", "mean_roi_area", "face_rate", "left_hand_rate", "right_hand_rate"]
camera_stats["fallback_pct"] = (camera_stats["fallback_pct"] * 100).round(1)
camera_stats["face_rate"] = (camera_stats["face_rate"] * 100).round(1)
camera_stats["left_hand_rate"] = (camera_stats["left_hand_rate"] * 100).round(1)
camera_stats["right_hand_rate"] = (camera_stats["right_hand_rate"] * 100).round(1)
print(camera_stats.to_string())

print("\n📋 BREAKDOWN BY CLASS")
print("-" * 80)
class_stats = df.groupby("class_id").agg({
    "fallback_to_full": ["sum", "mean"],
    "roi_area_frac": "mean",
    "face_count": lambda x: (x > 0).mean(),
    "left_hand_detected": "mean",
    "right_hand_detected": "mean",
}).round(3)
class_stats.columns = ["fallback_count", "fallback_pct", "mean_roi_area", "face_rate", "left_hand_rate", "right_hand_rate"]
class_stats["fallback_pct"] = (class_stats["fallback_pct"] * 100).round(1)
class_stats["face_rate"] = (class_stats["face_rate"] * 100).round(1)
class_stats["left_hand_rate"] = (class_stats["left_hand_rate"] * 100).round(1)
class_stats["right_hand_rate"] = (class_stats["right_hand_rate"] * 100).round(1)
print(class_stats.to_string())


## 🧪 Face-Only / Hands-Only Ablation Setup

**Purpose:** Understand which modality (face vs hands) carries the signal for each class.

**This is NOT about "winning accuracy"** - these ablations will usually be worse than face+hands combined. The goal is to produce **evidence** about:
1. Which modality explains which classes (hands-dominated vs face-dominated)
2. What types of errors happen when a modality is removed  
3. Whether Grad-CAM shows meaningful cues vs shortcuts within each modality

**Workflow:**
1. Run **face-only extraction** (change `VARIANT = "face"` in cell above, re-run)
2. Copy to `/content/data/hybrid/face/` (or run from there directly)
3. Train a face-only model
4. Repeat for **hands-only** (`VARIANT = "hands"`)

**NEW: Automatic Fallback Skipping**
- `hands` variant: **ONLY saves images where hands were detected** (skips fallbacks)
- `face` variant: **ONLY saves images where face was detected** (skips fallbacks)
- The split CSVs automatically contain only the saved images
- No "zoomed out" full-frame images will contaminate your ablation data!

**Evaluation is automatic subset-only:**
- Face-only models train/eval on images where face detection succeeded
- Hands-only models train/eval on images where hand detection succeeded
- Check `detection_metadata_*.csv` for coverage stats


In [ ]:
# 🧪 Ablation Helper: Check Detection Coverage for Subset-Only Evaluation

import pandas as pd
from pathlib import Path

# Configuration
HYBRID_ROOT = Path(os.environ.get("HYBRID_ROOT_LOCAL", Path(OUT_ROOT) / "hybrid"))

print("="*60)
print("🧪 ABLATION SETUP: Detection Coverage Analysis")
print("="*60)

# Load detection metadata for face_hands (it includes both face and hands detection status)
metadata_files = list(HYBRID_ROOT.glob("detection_metadata_*.csv"))
if not metadata_files:
    print("⚠️  No detection metadata found. Run hybrid extraction first.")
else:
    for mf in metadata_files:
        variant = mf.stem.replace("detection_metadata_", "")
        print(f"\n📊 Variant: {variant}")
        
        df = pd.read_csv(mf)
        total = len(df)
        
        # Check for detection columns
        if "face_detected" in df.columns:
            face_ok = df["face_detected"].sum()
            print(f"   Face detected: {face_ok}/{total} ({100*face_ok/total:.1f}%)")
        
        if "hands_detected" in df.columns:
            hands_ok = df["hands_detected"].sum()
            print(f"   Hands detected: {hands_ok}/{total} ({100*hands_ok/total:.1f}%)")
        elif "left_hand_detected" in df.columns and "right_hand_detected" in df.columns:
            any_hand = (df["left_hand_detected"] | df["right_hand_detected"]).sum()
            print(f"   Any hand detected: {any_hand}/{total} ({100*any_hand/total:.1f}%)")
        
        # If using hybrid with face_hands, calculate fallback rate
        if "used_fallback" in df.columns:
            fallback = df["used_fallback"].sum()
            print(f"   Used fallback (full frame): {fallback}/{total} ({100*fallback/total:.1f}%)")

print("\n" + "="*60)
print("📋 ABLATION VARIANTS TO TRAIN:")
print("="*60)
print("""
To run face-only or hands-only ablations:

1. EXTRACTION (modify cell above):
   VARIANT = "face"   # or "hands" 
   Re-run the hybrid extraction cell

2. COPY to /content:
   Change SRC_VARIANT_DIR and DST_ROOT in the copy cell

3. TRAINING:
   Create a new training run with appropriate tag:
   TRAIN_TAG = "effb0_face_only_V1"
   # or
   TRAIN_TAG = "effb0_hands_only_V1"

4. EVALUATION (subset-only):
   Use the detection metadata to filter predictions to only
   images where the relevant modality was detected.
""")

# Show which ablation datasets are available
print("\n📁 Available ablation datasets:")
for variant in ["face", "hands", "face_hands"]:
    variant_dir = HYBRID_ROOT / variant
    if variant_dir.exists():
        n_files = len(list(variant_dir.glob("**/*.jpg")))
        print(f"   ✅ {variant}: {n_files} images")
    else:
        print(f"   ⬜ {variant}: not extracted yet")


## 🚂 11.1 Train a model (adjust these knobs)

- Choose a `RUN_TAG` so logs/checkpoints go into `TFM/checkpoints/runs/<tag>/...`
- Set epochs/batch size to something small for a dry run (1 epoch, 16 batch)
- This command uses the CLI helper (`python -m src.ddriver.cli.train ...`)
- Logs + checkpoints are saved automatically to Google Drive


In [ ]:
import os
import subprocess, textwrap, json, time, threading
from pathlib import Path

# ConvNeXt-Tiny baseline run (change RUN_TAG for each experiment)
RUN_TAG = "effb0_noLabelSmoothingCORRECTED"   # change me for each experiment
MODEL_NAME = "efficientnet_b0"                  # must be registered above (timm)

# Training hyperparameters (EfficientNet-B0)
EPOCHS = 15
BATCH_SIZE = 32
NUM_WORKERS = 2
IMAGE_SIZE = 224
LR = 3e-4                        # per provided hyperparams
LR_DROP_EPOCH = None             # no LR drop
LR_DROP_FACTOR = 0.1
LABEL_SMOOTHING = 0.0
USE_TINY_SPLIT = False

# ROI crop pipeline selection (pick ONE, set others to False)
USE_MEDIAPIPE = False             # set True to use MediaPipe ROI crops
USE_YOLO = False                  # set True to use YOLO-World ROI crops
USE_HYBRID = True                 # set True to use Hybrid (RetinaFace + MediaPipe Hands) crops
ROI_VARIANT = "face_hands"        # face | hands | face_hands

# Validate only one pipeline is selected
active_pipelines = sum([USE_MEDIAPIPE, USE_YOLO, USE_HYBRID])
if active_pipelines > 1:
    raise ValueError("Pick ONE pipeline: set only one of USE_MEDIAPIPE, USE_YOLO, USE_HYBRID to True.")

if USE_HYBRID:
    hybrid_root = Path(os.environ.get("HYBRID_ROOT_LOCAL", Path(OUT_ROOT) / "hybrid"))
    manifest_csv = hybrid_root / f"manifest_{ROI_VARIANT}.csv"
    train_split = f"train_{ROI_VARIANT}.csv" if not USE_TINY_SPLIT else f"train_small_{ROI_VARIANT}.csv"
    val_csv = hybrid_root / f"val_{ROI_VARIANT}.csv"
    test_csv = hybrid_root / f"test_{ROI_VARIANT}.csv"
    train_csv = hybrid_root / train_split
    print(f"🔀 Using Hybrid (RetinaFace + MediaPipe Hands) ROI variant: {ROI_VARIANT}")
    print(f"   hybrid_root = {hybrid_root}")
elif USE_YOLO:
    yolo_root = Path(os.environ.get("YOLO_ROOT_LOCAL", Path(OUT_ROOT) / "yolo"))
    manifest_csv = yolo_root / f"manifest_{ROI_VARIANT}.csv"
    train_split = f"train_{ROI_VARIANT}.csv" if not USE_TINY_SPLIT else f"train_small_{ROI_VARIANT}.csv"
    val_csv = yolo_root / f"val_{ROI_VARIANT}.csv"
    test_csv = yolo_root / f"test_{ROI_VARIANT}.csv"
    train_csv = yolo_root / train_split
    print(f"🎯 Using YOLO-World ROI variant: {ROI_VARIANT}")
    print(f"   yolo_root = {yolo_root}")
elif USE_MEDIAPIPE:
    mp_root = Path(os.environ.get("MEDIAPIPE_ROOT_LOCAL", Path(OUT_ROOT) / "mediapipe"))
    manifest_csv = mp_root / f"manifest_{ROI_VARIANT}.csv"
    train_split = f"train_{ROI_VARIANT}.csv" if not USE_TINY_SPLIT else f"train_small_{ROI_VARIANT}.csv"
    val_csv = mp_root / f"val_{ROI_VARIANT}.csv"
    test_csv = mp_root / f"test_{ROI_VARIANT}.csv"
    train_csv = mp_root / train_split
    print(f"🧭 Using MediaPipe ROI variant: {ROI_VARIANT}")
    print(f"   mp_root = {mp_root}")
else:
    manifest_csv = Path(OUT_ROOT) / "manifests" / "manifest.csv"
    train_split = "train_small.csv" if USE_TINY_SPLIT else "train.csv"
    train_csv = Path(OUT_ROOT) / "splits" / train_split
    val_csv = Path(OUT_ROOT) / "splits" / "val.csv"
    test_csv = Path(OUT_ROOT) / "splits" / "test.csv"
    print("📷 Using full-frame images (no ROI cropping)")

# CRITICAL: Update DATASET_ROOT when using local ROI crops so images load from fast SSD
_roi_root = None
if USE_HYBRID:
    _roi_root = hybrid_root
elif USE_YOLO:
    _roi_root = yolo_root
elif USE_MEDIAPIPE:
    _roi_root = mp_root

if _roi_root and str(_roi_root).startswith("/content/data"):
    import importlib
    os.environ["DATASET_ROOT"] = str(_roi_root)
    from ddriver import config as _cfg
    importlib.reload(_cfg)
    print(f"   ⚡ DATASET_ROOT = {os.environ['DATASET_ROOT']} (fast local SSD)")
elif _roi_root:
    print(f"   ⚠️  DATASET_ROOT = {os.environ.get('DATASET_ROOT', 'not set')} (reading from Drive - may be slow)")

if USE_TINY_SPLIT:
    print("⚡ Using train_small.csv (20 imgs/class) for a quick smoke test.")
else:
    print("🪵 Using full train.csv for a proper run.")

train_cmd = textwrap.dedent(f"""
cd {PROJECT_ROOT}
python -m src.ddriver.cli.train \
    --model-name {MODEL_NAME} \
    --epochs {EPOCHS} \
    --batch-size {BATCH_SIZE} \
    --num-workers {NUM_WORKERS} \
    --image-size {IMAGE_SIZE} \
    --lr {LR} \
    --weight-decay 0.0 \
    --optimizer adam \
    --label-smoothing {LABEL_SMOOTHING} \
    --out-tag {RUN_TAG} \
    --manifest-csv {manifest_csv} \
    --train-csv {train_csv} \
    --val-csv {val_csv} \
    --test-csv {test_csv}
""")

print("Running training command and streaming logs:\n", train_cmd)

proc = subprocess.Popen(
    train_cmd,
    shell=True,
    text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
)

# Background GPU monitor (prints every 5 seconds)
def _gpu_monitor():
    while proc.poll() is None:
        try:
            stats = subprocess.check_output(
                "nvidia-smi --query-gpu=utilization.gpu,memory.used,memory.total "
                "--format=csv,nounits,noheader",
                shell=True,
            ).decode("utf-8").strip()
            print(f"[GPU] util%, mem_used, mem_total :: {stats}")
        except Exception as exc:
            print("[GPU] Could not query nvidia-smi:", exc)
        time.sleep(5)

monitor_thread = threading.Thread(target=_gpu_monitor, daemon=True)
monitor_thread.start()

# Stream CLI stdout live
if proc.stdout is None:
    raise RuntimeError("Training process has no stdout pipe.")

for line in proc.stdout:
    print(line, end="")

proc.wait()
monitor_thread.join(timeout=0)

print("\n✅ Training run complete!\n")

# --- Display every epoch's metrics so the notebook shows the learning curve ---
run_base = Path(CKPT_ROOT) / "runs" / RUN_TAG
all_runs = sorted(run_base.glob("*/"))
if not all_runs:
    raise FileNotFoundError(f"No run folders found under {run_base}")
latest_run = all_runs[-1]

history_path = latest_run / "history.json"
if not history_path.exists():
    raise FileNotFoundError(f"history.json not found in {latest_run}")

history = json.loads(history_path.read_text()).get("history", [])
print(f"📊 Epoch metrics for run: {latest_run.name}")
for record in history:
    train_metrics = record.get("train", {})
    val_metrics = record.get("val", {}) or {}
    train_loss = train_metrics.get("loss")
    train_acc = train_metrics.get("accuracy")
    val_loss = val_metrics.get("loss")
    val_acc = val_metrics.get("accuracy")
    val_str = (
        f"val_loss={val_loss:.4f} acc={val_acc:.4f}"
        if val_loss is not None and val_acc is not None
        else "val_loss=— val_acc=—"
    )
    print(
        f"  Epoch {record['epoch']:>2}: "
        f"train_loss={train_loss:.4f} acc={train_acc:.4f}  "
        f"{val_str}"
    )
print("")


## 📝 11.1a Log training summary to Google Sheet
Run this right after the training cell finishes. It looks up the newest run under `CKPT_ROOT/runs/<RUN_TAG>`, grabs the best/final train + val accuracies, and logs the model/hyperparams so you can compare experiments before doing predictions or metrics.


## 🚚 Copy MediaPipe crops to /content (optional, faster I/O)
Use this if training from Drive is slow. It copies the generated MediaPipe crops/CSVs into `/content/data/mediapipe/<variant>` and updates paths for the current runtime. Does not affect the original full-image copy cell.


In [ ]:
import os, shutil
from pathlib import Path

# Set your variant to the one you extracted already
MEDIAPIPE_VARIANT = "face_hands"  # face | hands | face_hands
SRC_ROOT = Path(OUT_ROOT) / "mediapipe"
SRC_VARIANT_DIR = SRC_ROOT / MEDIAPIPE_VARIANT
DST_ROOT = Path("/content/data/mediapipe") / MEDIAPIPE_VARIANT

if not SRC_VARIANT_DIR.exists():
    raise FileNotFoundError(f"Source mediapipe folder not found: {SRC_VARIANT_DIR}\nRun the extraction cell first.")

print(f"Copying MediaPipe crops from {SRC_VARIANT_DIR} -> {DST_ROOT}")
DST_ROOT.mkdir(parents=True, exist_ok=True)

# Copy images directory tree
for src_dir, _, files in os.walk(SRC_VARIANT_DIR):
    rel_dir = Path(src_dir).relative_to(SRC_VARIANT_DIR)
    dst_dir = DST_ROOT / rel_dir
    dst_dir.mkdir(parents=True, exist_ok=True)
    for fname in files:
        src_path = Path(src_dir) / fname
        dst_path = dst_dir / fname
        if not dst_path.exists():
            shutil.copy2(src_path, dst_path)

# Copy manifest/split CSVs that live one level above the variant folder
csv_names = [
    f"manifest_{MEDIAPIPE_VARIANT}.csv",
    f"train_{MEDIAPIPE_VARIANT}.csv",
    f"val_{MEDIAPIPE_VARIANT}.csv",
    f"test_{MEDIAPIPE_VARIANT}.csv",
]
for fname in csv_names:
    src_csv = SRC_ROOT / fname
    if not src_csv.exists():
        raise FileNotFoundError(src_csv)
    dst_csv = DST_ROOT.parent / fname
    shutil.copy2(src_csv, dst_csv)
    print(f"Copied {src_csv} -> {dst_csv}")

# Point env vars for this runtime to the local mediapipe copy
os.environ["MEDIAPIPE_ROOT_LOCAL"] = str(DST_ROOT.parent)
os.environ["MEDIAPIPE_VARIANT"] = MEDIAPIPE_VARIANT

# CRITICAL: Also update DATASET_ROOT so the Dataset class loads images from local copy!
os.environ["DATASET_ROOT"] = str(DST_ROOT.parent)
import importlib
from ddriver import config as _cfg
importlib.reload(_cfg)

print("\n✅ Copy complete. Set USE_MEDIAPIPE=True in training cell.")
print(f"   MEDIAPIPE_ROOT_LOCAL = {os.environ['MEDIAPIPE_ROOT_LOCAL']}")
print(f"   DATASET_ROOT         = {os.environ['DATASET_ROOT']} (images loaded from here)")



In [ ]:
# 📝 Training summary → Google Sheet
!pip -q install gspread

import json
from pathlib import Path

import gspread
from google.colab import auth
import google.auth

auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

TRAIN_SHEET_NAME = "TFM Train Logs"   # create this sheet/tab ahead of time
TRAIN_WORKSHEET = "Sheet1"

run_base = Path(CKPT_ROOT) / "runs" / RUN_TAG
all_runs = sorted(run_base.glob("*/"))
if not all_runs:
    raise FileNotFoundError(f"No run folders found under {run_base}")
latest_run = all_runs[-1]
print(f"Logging training summary for run folder: {latest_run}")

history_path = latest_run / "history.json"
if not history_path.exists():
    raise FileNotFoundError(f"history.json not found under {latest_run}")

history_records = json.loads(history_path.read_text()).get("history", [])
if not history_records:
    raise ValueError(f"history.json under {latest_run} has no records.")

params_path = latest_run / "params.json"
params = json.loads(params_path.read_text()) if params_path.exists() else {}

model_name = params.get("model_name", MODEL_NAME)
epochs_cfg = params.get("epochs", EPOCHS)
batch_cfg = params.get("batch_size", BATCH_SIZE)
lr_cfg = params.get("lr", LR)
lr_drop_epoch_cfg = params.get("lr_drop_epoch", LR_DROP_EPOCH)
lr_drop_factor_cfg = params.get("lr_drop_factor", LR_DROP_FACTOR)
image_size_cfg = params.get("image_size", IMAGE_SIZE)
num_workers_cfg = params.get("num_workers", NUM_WORKERS)
use_tiny_cfg = params.get("use_tiny_split", USE_TINY_SPLIT)


def _best_metric(records, split: str) -> tuple[dict, float | None]:
    best_epoch = None
    best_acc = None
    for rec in records:
        split_metrics = rec.get(split) or {}
        acc = split_metrics.get("accuracy")
        if acc is None:
            continue
        if best_acc is None or acc > best_acc:
            best_acc = acc
            best_epoch = rec.get("epoch")
    final_metrics = records[-1].get(split) or {}
    final_acc = final_metrics.get("accuracy")
    return {"epoch": best_epoch, "accuracy": best_acc}, final_acc


best_train, final_train = _best_metric(history_records, "train")
best_val, final_val = _best_metric(history_records, "val")

row = [
    RUN_TAG,
    latest_run.name,
    model_name,
    epochs_cfg,
    batch_cfg,
    lr_cfg,
    lr_drop_epoch_cfg,
    lr_drop_factor_cfg,
    image_size_cfg,
    num_workers_cfg,
    use_tiny_cfg,
    best_train["epoch"] if best_train["epoch"] is not None else "",
    round(best_train["accuracy"], 4) if best_train["accuracy"] is not None else "",
    best_val["epoch"] if best_val["epoch"] is not None else "",
    round(best_val["accuracy"], 4) if best_val["accuracy"] is not None else "",
    round(final_train, 4) if final_train is not None else "",
    round(final_val, 4) if final_val is not None else "",
]

ws = gc.open(TRAIN_SHEET_NAME).worksheet(TRAIN_WORKSHEET)
ws.append_row(row, value_input_option="USER_ENTERED")
print(f"Appended training summary for {latest_run.name} ✅")



In [ ]:
# 🔄 Optional: copy + compress dataset subset → fast local SSD (/content/data)
# Re-encodes JPEGs once (quality 80, short side 320px) before landing in /content/data.

import importlib
import os
from pathlib import Path

from ddriver.data.fastcopy import CompressionSpec, copy_splits_with_compression

SRC_ROOT = Path(DRIVE_DATA_ROOT) / "auc.distracted.driver.dataset_v2"
DST_ROOT = Path(FAST_DATA) / "auc.distracted.driver.dataset_v2"

split_csvs = {
    "train": Path(OUT_ROOT) / "splits" / "train.csv",
    "val": Path(OUT_ROOT) / "splits" / "val.csv",
    "train_small": Path(OUT_ROOT) / "splits" / "train_small.csv",
}

compression_spec = CompressionSpec(
    target_short_side=320,  # still >= image_size + resize margin for training
    jpeg_quality=80,        # ImageNet-level compression, visually lossless
)

summary = copy_splits_with_compression(
    split_csvs=split_csvs,
    src_root=SRC_ROOT,
    dst_root=DST_ROOT,
    compression=compression_spec,
    skip_existing=True,
)

print(
    f"\n📉 FAST_DATA copy stats: processed {summary['processed']} of {summary['total']} files "
    f"(skipped {summary['skipped']} already present)."
)
print(f"Compressed dataset root: {summary['dst_root']}")

DATASET_ROOT = FAST_DATA
os.environ["DATASET_ROOT"] = str(DATASET_ROOT)
try:
    from ddriver import config as _ddriver_config
    importlib.reload(_ddriver_config)
    print("\n⚡ Copy complete. DATASET_ROOT now points to the local FAST_DATA copy for this runtime:")
    print("   ddriver.config.DATASET_ROOT =", _ddriver_config.DATASET_ROOT)
except Exception as exc:
    print("\n⚡ Copy complete and DATASET_ROOT env updated, but could not reload ddriver.config:", exc)
print("   (Re-run env summary if you want to rewrite .env, but training now uses /content/data.)")


In [ ]:
# 🔄 Optional: copy + compress TEST split → /content/data (same settings as train/val)

import importlib
from pathlib import Path

from ddriver.data.fastcopy import CompressionSpec, copy_splits_with_compression

SRC_ROOT = Path(DRIVE_DATA_ROOT) / "auc.distracted.driver.dataset_v2"
DST_ROOT = Path(FAST_DATA) / "auc.distracted.driver.dataset_v2"

split_csvs = {
    "test": Path(OUT_ROOT) / "splits" / "test.csv",
}

compression_spec = CompressionSpec(
    target_short_side=320,
    jpeg_quality=80,
)

summary = copy_splits_with_compression(
    split_csvs=split_csvs,
    src_root=SRC_ROOT,
    dst_root=DST_ROOT,
    compression=compression_spec,
    skip_existing=True,
)

print(
    f"\n📉 FAST_DATA test copy stats: processed {summary['processed']} of {summary['total']} "
    f"(skipped {summary['skipped']} already present)."
)
print(f"Compressed dataset root: {summary['dst_root']}")

# DATASET_ROOT is already pointing at FAST_DATA from the earlier cell, but reload config just in case
try:
    from ddriver import config as _ddriver_config
    importlib.reload(_ddriver_config)
    print("\n⚡ Test copy complete. ddriver.config now sees:")
    print("   ddriver.config.DATASET_ROOT =", _ddriver_config.DATASET_ROOT)
except Exception as exc:
    print("\n⚡ Test copy complete; config reload optional:", exc)

## 📦 11.2 Pick the latest checkpoint file

This cell looks inside `CKPT_ROOT/runs/<RUN_TAG>/` and grabs the newest `epoch_*.pt`. Use this path in the prediction step.


In [ ]:
from pathlib import Path

RUN_TAG = "convnext_tiny_full_v1"  # pick the tag you want to inspect
#RUN_TAG = globals().get("RUN_TAG", "convnext_tiny_full_v1")  # reuse your latest training tag by default

run_base = Path(CKPT_ROOT) / "runs" / RUN_TAG
runs = sorted(run_base.glob("*/"))
if not runs:
    raise FileNotFoundError(f"No run folders found under {run_base}")

# ---- choose which run folder to use ----
RUN_IDX = -1          # -1 = newest, 0 = oldest, or any index from the printout below
print("Available runs:")
for idx, run_dir in enumerate(runs):
    print(f"  [{idx}] {run_dir.name}")
target_run = runs[RUN_IDX]
print(f"\nSelected run: {target_run}\n")

# ---- choose which checkpoint (epoch) inside that run ----
checkpoint_patterns = ["epoch_*.pt", "best.pt", "last.pt"]
checkpoints = []
for pattern in checkpoint_patterns:
       matches = sorted(target_run.glob(pattern))
       if matches:
           checkpoints.extend(matches)

if not checkpoints:
       raise FileNotFoundError(f"No checkpoints found under {target_run}")

CHECKPOINT_NAME = "best.pt"  # or "last.pt", or None to take the last match
if CHECKPOINT_NAME:
       chosen_ckpt = target_run / CHECKPOINT_NAME
       if not chosen_ckpt.exists():
           raise FileNotFoundError(chosen_ckpt)
else:
       chosen_ckpt = checkpoints[-1]

LATEST_CKPT = chosen_ckpt
print("Using checkpoint:", LATEST_CKPT)

## 🔮 11.3 Generate predictions CSV

- Uses the checkpoint above
- Choose which split to predict on (`val` or `test`)
- Saves CSV under `OUT_ROOT/preds/<split>/<out_tag>.csv`


In [ ]:
import os
import subprocess
import textwrap
from pathlib import Path

# ============================================================================
# 🔧 PREDICTION SETTINGS - Configure these for inference-only runs
# ============================================================================
PRED_SPLIT = "test"           # or "val"

# Set to True and specify variant if using ROI-cropped data:
USE_HYBRID = True             # Set False for full-image models
# USE_YOLO = False            # Uncomment if using YOLO crops
# USE_MEDIAPIPE = False       # Uncomment if using MediaPipe crops
ROI_VARIANT = "face_hands"    # e.g., "face_hands", "face", "hands"

# Hyperparameters (can be different from training for inference):
BATCH_SIZE = 32               # Can be larger for inference (no gradients)
NUM_WORKERS = 2               # Adjust based on your machine
IMAGE_SIZE = 224              # Must match what the model was trained with

# Model name - must match the architecture in your checkpoint:
MODEL_NAME = "efficientnet_b0"  # e.g., "convnext_tiny", "resnet50", etc.

# Infer RUN_TAG from checkpoint path if not already set:
if "RUN_TAG" not in dir() or RUN_TAG is None:
    # e.g., .../runs/convnext_tiny_full_v1/2024-12-14/best.pt -> convnext_tiny_full_v1
    RUN_TAG = LATEST_CKPT.parent.parent.name
    print(f"Inferred RUN_TAG from checkpoint: {RUN_TAG}")

PRED_TAG = f"{RUN_TAG}_{PRED_SPLIT}"

# ============================================================================
# Build manifest/split paths based on data source
# ============================================================================
if "USE_HYBRID" in globals() and USE_HYBRID:
    hybrid_root = Path(os.environ.get("HYBRID_ROOT_LOCAL", Path(OUT_ROOT) / "hybrid"))
    manifest_pred = hybrid_root / f"manifest_{ROI_VARIANT}.csv"
    train_pred = hybrid_root / f"train_{ROI_VARIANT}.csv"
    val_pred = hybrid_root / f"val_{ROI_VARIANT}.csv"
    test_pred = hybrid_root / f"test_{ROI_VARIANT}.csv"
    manifest_flag = f"--manifest-csv {manifest_pred}"
    split_flag_str = f"--train-csv {train_pred} --val-csv {val_pred} --test-csv {test_pred}"
    print(f"📦 Using HYBRID crops: {ROI_VARIANT}")
elif "USE_YOLO" in globals() and USE_YOLO:
    yolo_root = Path(os.environ.get("YOLO_ROOT_LOCAL", Path(OUT_ROOT) / "yolo"))
    manifest_pred = yolo_root / f"manifest_{ROI_VARIANT}.csv"
    train_pred = yolo_root / f"train_{ROI_VARIANT}.csv"
    val_pred = yolo_root / f"val_{ROI_VARIANT}.csv"
    test_pred = yolo_root / f"test_{ROI_VARIANT}.csv"
    manifest_flag = f"--manifest-csv {manifest_pred}"
    split_flag_str = f"--train-csv {train_pred} --val-csv {val_pred} --test-csv {test_pred}"
    print(f"📦 Using YOLO crops: {ROI_VARIANT}")
elif "USE_MEDIAPIPE" in globals() and USE_MEDIAPIPE:
    mp_root = Path(os.environ.get("MEDIAPIPE_ROOT_LOCAL", Path(OUT_ROOT) / "mediapipe"))
    manifest_pred = mp_root / f"manifest_{ROI_VARIANT}.csv"
    train_pred = mp_root / f"train_{ROI_VARIANT}.csv"
    val_pred = mp_root / f"val_{ROI_VARIANT}.csv"
    test_pred = mp_root / f"test_{ROI_VARIANT}.csv"
    manifest_flag = f"--manifest-csv {manifest_pred}"
    split_flag_str = f"--train-csv {train_pred} --val-csv {val_pred} --test-csv {test_pred}"
    print(f"📦 Using MEDIAPIPE crops: {ROI_VARIANT}")
else:
    manifest_pred = Path(OUT_ROOT) / "manifests" / "manifest.csv"
    train_pred = Path(OUT_ROOT) / "splits" / "train.csv"
    val_pred = Path(OUT_ROOT) / "splits" / "val.csv"
    test_pred = Path(OUT_ROOT) / "splits" / "test.csv"
    manifest_flag = f"--manifest-csv {manifest_pred}"
    split_flag_str = f"--train-csv {train_pred} --val-csv {val_pred} --test-csv {test_pred}"
    print("📦 Using FULL images (no ROI cropping)")

# ============================================================================
# CRITICAL: Set DATASET_ROOT to point to the correct data folder
# For hybrid/YOLO/MediaPipe, images are in the crop output folder, not original data
# ============================================================================
import importlib
from ddriver import config as _cfg

if "USE_HYBRID" in globals() and USE_HYBRID:
    # Use fast local copy if available (HYBRID_ROOT_LOCAL), otherwise fall back to Drive
    _new_dataset_root = os.environ.get("HYBRID_ROOT_LOCAL", str(Path(OUT_ROOT) / "hybrid"))
    os.environ["DATASET_ROOT"] = _new_dataset_root
    importlib.reload(_cfg)
    print(f"📂 DATASET_ROOT → {_new_dataset_root} (hybrid crops)")
elif "USE_YOLO" in globals() and USE_YOLO:
    # Use fast local copy if available
    _new_dataset_root = os.environ.get("YOLO_ROOT_LOCAL", str(Path(OUT_ROOT) / "yolo"))
    os.environ["DATASET_ROOT"] = _new_dataset_root
    importlib.reload(_cfg)
    print(f"📂 DATASET_ROOT → {_new_dataset_root} (YOLO crops)")
elif "USE_MEDIAPIPE" in globals() and USE_MEDIAPIPE:
    # Use fast local copy if available
    _new_dataset_root = os.environ.get("MEDIAPIPE_ROOT_LOCAL", str(Path(OUT_ROOT) / "mediapipe"))
    os.environ["DATASET_ROOT"] = _new_dataset_root
    importlib.reload(_cfg)
    print(f"📂 DATASET_ROOT → {_new_dataset_root} (MediaPipe crops)")
else:
    print(f"📂 DATASET_ROOT = {os.environ.get('DATASET_ROOT', 'not set')} (original images)")

predict_cmd = textwrap.dedent(f"""
cd {PROJECT_ROOT}
python -m src.ddriver.cli.predict \
    --model-name {MODEL_NAME} \
    --checkpoint {LATEST_CKPT} \
    --split {PRED_SPLIT} \
    --batch-size {BATCH_SIZE} \
    --num-workers {NUM_WORKERS} \
    --image-size {IMAGE_SIZE} \
    --out-tag {PRED_TAG} \
    {manifest_flag} \
    {split_flag_str}
""")

print("Running prediction command:\n", predict_cmd)
result = subprocess.run(predict_cmd, shell=True, text=True, capture_output=True)
if result.returncode != 0:
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    raise RuntimeError("Prediction command failed. See logs above.")
print(result.stdout)
print("\n✅ Predictions completed! Check OUT_ROOT/preds/")


## 📊 11.4 Evaluate metrics

- Uses `src/ddriver/metrics.py`
- Reads the manifest + split CSV + predictions CSV
- Saves results under `OUT_ROOT/metrics/<tag>/<timestamp>/`
- Shows accuracy + macro F1 + per-driver/camera (optional)


In [ ]:
import os
import subprocess
import textwrap
from pathlib import Path

# Uses settings from prediction cell (PRED_SPLIT, PRED_TAG, USE_HYBRID, ROI_VARIANT, etc.)
# If you skipped prediction cell, set them here:
# PRED_SPLIT = "test"
# PRED_TAG = "your_run_tag_test"
# USE_HYBRID = True
# ROI_VARIANT = "face_hands"

if "USE_HYBRID" in globals() and USE_HYBRID:
    hybrid_root = Path(os.environ.get("HYBRID_ROOT_LOCAL", Path(OUT_ROOT) / "hybrid"))
    manifest_path = hybrid_root / f"manifest_{ROI_VARIANT}.csv"
    split_csv_path = hybrid_root / f"{PRED_SPLIT}_{ROI_VARIANT}.csv"
    preds_csv_path = Path(OUT_ROOT) / "preds" / PRED_SPLIT / f"{PRED_TAG}.csv"
elif "USE_YOLO" in globals() and USE_YOLO:
    yolo_root = Path(os.environ.get("YOLO_ROOT_LOCAL", Path(OUT_ROOT) / "yolo"))
    manifest_path = yolo_root / f"manifest_{ROI_VARIANT}.csv"
    split_csv_path = yolo_root / f"{PRED_SPLIT}_{ROI_VARIANT}.csv"
    preds_csv_path = Path(OUT_ROOT) / "preds" / PRED_SPLIT / f"{PRED_TAG}.csv"
elif "USE_MEDIAPIPE" in globals() and USE_MEDIAPIPE:
    mp_root = Path(os.environ.get("MEDIAPIPE_ROOT_LOCAL", Path(OUT_ROOT) / "mediapipe"))
    manifest_path = mp_root / f"manifest_{ROI_VARIANT}.csv"
    split_csv_path = mp_root / f"{PRED_SPLIT}_{ROI_VARIANT}.csv"
    preds_csv_path = Path(OUT_ROOT) / "preds" / PRED_SPLIT / f"{PRED_TAG}.csv"
else:
    manifest_path = Path(OUT_ROOT) / "manifests" / "manifest.csv"
    split_csv_path = Path(OUT_ROOT) / "splits" / f"{PRED_SPLIT}.csv"
    preds_csv_path = Path(OUT_ROOT) / "preds" / PRED_SPLIT / f"{PRED_TAG}.csv"
METRICS_TAG = PRED_TAG

print(f"📊 Evaluating: {preds_csv_path}")

metrics_cmd = textwrap.dedent(f"""
cd {PROJECT_ROOT}
python -m src.ddriver.eval.metrics \
    --manifest {manifest_path} \
    --split-csv {split_csv_path} \
    --predictions {preds_csv_path} \
    --out-tag {METRICS_TAG} \
    --per-driver \
    --per-camera
""")

print("Running metrics command:\n", metrics_cmd)
result = subprocess.run(metrics_cmd, shell=True, text=True, capture_output=True)
if result.returncode != 0:
    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)
    raise RuntimeError("Metrics command failed. See logs above.")
print(result.stdout)
print("\n✅ Metrics saved under OUT_ROOT/metrics/")


### ✅ You're all set!

**What just happened:**
1. ✅ Mounted Google Drive
2. ✅ Cloned/updated your repo
3. ✅ Installed the package
4. ✅ Set up paths (works on Colab and Mac!)
5. ✅ Generated manifest.csv and train/val/test split CSVs
6. ✅ Tested that dataset.py can load images
7. ✅ Tested that datamod.py can create data loaders
8. ✅ (Optional) Registered a model + ran training → prediction → metrics pipeline

**Your CSVs are saved in Google Drive:**
- `OUT_ROOT/manifests/manifest.csv` - Big list of all images
- `OUT_ROOT/splits/train.csv` - Training images
- `OUT_ROOT/splits/val.csv` - Validation images (with driver IDs!)
- `OUT_ROOT/splits/test.csv` - Test images

**Next steps:**
- Adjust the training/prediction cells (epochs, batch size, tags) to run bigger experiments
- All paths use `ddriver.config` so it works on Colab and Mac
- Re-run **Clone/Update** cell after pushing new commits
- Optional: copy some data into `/content/data` to use `FAST_DATA` for speed


In [ ]:
# ---- Colab cell: append metrics + params to Google Sheet ----
!pip -q install gspread

import json
from pathlib import Path

import gspread
from google.colab import auth
import google.auth

auth.authenticate_user()
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

EVAL_SHEET_NAME = "TFM Eval Logs"   # create this sheet/tab ahead of time
EVAL_WORKSHEET = "Sheet1"

METRICS_TAG = (
    globals().get("METRICS_TAG")
    or globals().get("PRED_TAG")
    or "convnext_tiny_full_v1_val"
)  # match the --out-tag you used
metrics_root = Path(OUT_ROOT) / "metrics" / METRICS_TAG
runs = sorted(metrics_root.glob("*/"))
if not runs:
    raise FileNotFoundError(f"No metrics runs found under {metrics_root}")
latest_metrics = runs[-1]
print("Logging metrics folder:", latest_metrics)

def _read_json(path: Path, *, required: bool = True) -> dict:
    if not path.exists():
        if required:
            raise FileNotFoundError(f"Expected file missing: {path}")
        return {}
    return json.loads(path.read_text())

metrics = _read_json(latest_metrics / "metrics.json")
inputs = _read_json(latest_metrics / "inputs.json", required=False)
params = _read_json(latest_metrics / "params.json", required=False)

overall = metrics.get("overall", {})
macro = overall.get("macro_avg", {})

row = [
    str(latest_metrics),
    inputs.get("predictions", ""),
    inputs.get("split_source", ""),
    metrics.get("num_examples", ""),
    round(overall.get("accuracy", 0.0), 4),
    round(macro.get("f1", 0.0), 4),
    json.dumps(params, sort_keys=True)[:500],
]

ws = gc.open(EVAL_SHEET_NAME).worksheet(EVAL_WORKSHEET)
ws.append_row(row, value_input_option="USER_ENTERED")
print(f"Appended metrics run {latest_metrics.name} to {EVAL_SHEET_NAME}/{EVAL_WORKSHEET} ✅")

### 📊 11.4a Visualize Confusion Matrix

Quick peek at where the model confuses classes using the most recent metrics run.


In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

METRICS_TAG = (
    globals().get("METRICS_TAG")
    or globals().get("PRED_TAG")
    or "convnext_tiny_full_v1_val"
)  # change if you used a different --out-tag
metrics_root = Path(OUT_ROOT) / "metrics" / METRICS_TAG
runs = sorted(metrics_root.glob("*/"))
if not runs:
    raise FileNotFoundError(f"No metrics runs found under {metrics_root}")
latest_metrics = runs[-1]
print("Reading confusion matrix from:", latest_metrics)

metrics = json.loads((latest_metrics / "metrics.json").read_text())
cm_info = metrics.get("confusion_matrix")
if not cm_info:
    raise ValueError("confusion_matrix missing from metrics.json")

labels = cm_info["rows_cols_labels"]
cm_df = pd.DataFrame(cm_info["matrix"], index=labels, columns=labels)

counts_path = latest_metrics / "confusion_matrix_counts.png"
plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.title(f"Confusion matrix – {METRICS_TAG}")
plt.ylabel("True class")
plt.xlabel("Predicted class")
plt.tight_layout()
plt.savefig(counts_path)
plt.show()
print("Saved counts heatmap to", counts_path)

cm_norm = cm_df.div(cm_df.sum(axis=1).replace(0, 1), axis=0)
norm_path = latest_metrics / "confusion_matrix_normalized.png"
plt.figure(figsize=(8, 6))
sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap="Blues")
plt.title(f"Normalized confusion matrix – {METRICS_TAG}")
plt.ylabel("True class")
plt.xlabel("Predicted class")
plt.tight_layout()
plt.savefig(norm_path)
plt.show()
print("Saved normalized heatmap to", norm_path)


## 🧪 11.4b Subset-Only Evaluation (For Ablations)

For face-only or hands-only models, evaluate **only on images where detection succeeded**.

This gives a fair comparison: "What can face-only do on the 90% of images where face detection worked?"

Use this cell after running predictions on a face-only or hands-only model.


In [ ]:
# 🧪 Subset-Only Evaluation for Ablation Models
# Filters predictions to only images where the relevant modality was detected

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================================
# CONFIGURATION - Set these based on which ablation you're evaluating
# ============================================================================
ABLATION_TYPE = "face"  # "face" or "hands" - which modality this model uses
PRED_CSV = None  # Set to path, or None to use the latest from OUT_ROOT

# Where is the detection metadata?
HYBRID_ROOT = Path(os.environ.get("HYBRID_ROOT_LOCAL", Path(OUT_ROOT) / "hybrid"))

# ============================================================================
# LOAD DETECTION METADATA
# ============================================================================
# Try to find metadata that has detection status per image
metadata_candidates = [
    HYBRID_ROOT / f"detection_metadata_face_hands.csv",  # Most complete
    HYBRID_ROOT / f"detection_metadata_{ABLATION_TYPE}.csv",
]

metadata_df = None
for mf in metadata_candidates:
    if mf.exists():
        metadata_df = pd.read_csv(mf)
        print(f"✅ Loaded detection metadata from: {mf}")
        print(f"   Columns: {list(metadata_df.columns)}")
        break

if metadata_df is None:
    print("❌ No detection metadata found. Cannot do subset-only evaluation.")
    print("   Run hybrid extraction first to generate detection_metadata_*.csv")
else:
    # ============================================================================
    # LOAD PREDICTIONS
    # ============================================================================
    if PRED_CSV is None:
        # Try to find the latest prediction file
        pred_dir = Path(OUT_ROOT) / "preds" / "test"
        pred_files = sorted(pred_dir.glob("*.csv"), key=lambda x: x.stat().st_mtime, reverse=True)
        if pred_files:
            PRED_CSV = pred_files[0]
            print(f"📄 Using latest predictions: {PRED_CSV}")
        else:
            raise FileNotFoundError(f"No prediction files found in {pred_dir}")
    
    preds_df = pd.read_csv(PRED_CSV)
    print(f"   Total predictions: {len(preds_df)}")
    
    # ============================================================================
    # FILTER TO DETECTED SUBSET
    # ============================================================================
    # Extract filename from prediction paths to match with metadata
    preds_df["_filename"] = preds_df["path"].apply(lambda p: Path(p).name)
    
    # Also extract from metadata
    if "source_path" in metadata_df.columns:
        metadata_df["_filename"] = metadata_df["source_path"].apply(lambda p: Path(p).name)
    elif "path" in metadata_df.columns:
        metadata_df["_filename"] = metadata_df["path"].apply(lambda p: Path(p).name)
    
    # Determine which column indicates detection success
    if ABLATION_TYPE == "face":
        det_col = "face_detected" if "face_detected" in metadata_df.columns else None
    else:  # hands
        if "hands_detected" in metadata_df.columns:
            det_col = "hands_detected"
        elif "left_hand_detected" in metadata_df.columns:
            metadata_df["hands_detected"] = metadata_df["left_hand_detected"] | metadata_df["right_hand_detected"]
            det_col = "hands_detected"
        else:
            det_col = None
    
    if det_col is None:
        print(f"❌ Cannot find {ABLATION_TYPE} detection column in metadata")
    else:
        # Get list of filenames where detection succeeded
        # Get list of filenames where detection succeeded
        detected_df = metadata_df[metadata_df[det_col] == True].copy()
        
        # ADDITIONAL FILTER: For hands-only or face-only, exclude crops that are too large
        # (they likely accidentally include other modalities due to over-padding)
        MAX_AREA_FRAC = 0.50  # Exclude crops > 50% of image (tune as needed)
        detected_df = detected_df[
            (detected_df["roi_area_frac"] <= MAX_AREA_FRAC) & 
            (detected_df["fallback_to_full"] == False)
        ]
        print(f"   Filtered: excluding crops with roi_area_frac > {MAX_AREA_FRAC}")
        print(f"   Filtered: excluding fallback_to_full=True")
        n_before = len(metadata_df[metadata_df[det_col] == True])
        n_after = len(detected_df)
        print(f"   After size filtering: {n_after}/{n_before} images ({100*n_after/n_before:.1f}%)")
        
        detected_filenames = set(detected_df["_filename"])
        # Filter predictions
        preds_subset = preds_df[preds_df["_filename"].isin(detected_filenames)]
        
        print(f"\n📊 SUBSET-ONLY EVALUATION ({ABLATION_TYPE.upper()})")
        print(f"="*50)
        print(f"   Full test set: {len(preds_df)} images")
        print(f"   {ABLATION_TYPE.title()} detected: {len(preds_subset)} images ({100*len(preds_subset)/len(preds_df):.1f}%)")
        
        # ============================================================================
        # COMPUTE METRICS ON SUBSET
        # ============================================================================
        # Need ground truth labels - extract from path or merge with manifest
        if "label" not in preds_subset.columns and "class_id" not in preds_subset.columns:
            # Extract class from path (assuming c0/, c1/, etc. in path)
            def extract_class(path):
                parts = Path(path).parts
                for p in reversed(parts):
                    if p.startswith("c") and len(p) == 2 and p[1].isdigit():
                        return int(p[1])
                return -1
            preds_subset = preds_subset.copy()
            preds_subset["label"] = preds_subset["path"].apply(extract_class)
        elif "class_id" in preds_subset.columns:
            preds_subset = preds_subset.copy()
            preds_subset["label"] = preds_subset["class_id"].apply(
                lambda x: int(x[1]) if isinstance(x, str) and x.startswith("c") else int(x)
            )
        
        # Convert predictions to int
        if "pred" not in preds_subset.columns:
            preds_subset["pred"] = preds_subset["pred_class_id"].apply(
                lambda x: int(x[1]) if isinstance(x, str) and x.startswith("c") else int(x)
            )
        
        # Compute accuracy
        valid_preds = preds_subset[preds_subset["label"] >= 0]
        accuracy = accuracy_score(valid_preds["label"], valid_preds["pred"])
        
        print(f"\n   SUBSET ACCURACY: {accuracy*100:.2f}%")
        print(f"\n   Per-class breakdown:")
        print(classification_report(valid_preds["label"], valid_preds["pred"], 
                                   target_names=[f"c{i}" for i in range(10)]))
        
        # Confusion matrix
        cm = confusion_matrix(valid_preds["label"], valid_preds["pred"])
        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax,
                   xticklabels=[f"c{i}" for i in range(10)],
                   yticklabels=[f"c{i}" for i in range(10)])
        ax.set_xlabel("Predicted")
        ax.set_ylabel("True")
        ax.set_title(f"Confusion Matrix: {ABLATION_TYPE.title()}-Only Model (Subset Evaluation)\n"
                    f"Accuracy: {accuracy*100:.1f}% on {len(valid_preds)}/{len(preds_df)} images")
        plt.tight_layout()
        
        # Save
        out_path = Path(OUT_ROOT) / "metrics" / f"confusion_{ABLATION_TYPE}_only_subset.png"
        out_path.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(out_path, dpi=150)
        plt.show()
        print(f"\n💾 Saved to {out_path}")


## 📊 11.4c Per-Class Modality Comparison (Optional)

Compare face-only, hands-only, and face+hands performance **per class** to answer:
- Which classes are **hands-dominated**? (texting, drinking, radio)
- Which classes are **face-dominated**? (talking to passenger, phone call)
- Which classes need **both modalities**?

**Prerequisites:** Run predictions for all three model variants first.


In [ ]:
# 📊 Per-Class Modality Comparison: Face vs Hands vs Face+Hands

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# ============================================================================
# CONFIGURATION - Set prediction file paths for each model variant
# ============================================================================
# Set these to your actual prediction CSV files (or None if not available yet)
FACE_ONLY_PREDS = None  # e.g., Path(OUT_ROOT) / "preds/test/effb0_face_only_V1_test.csv"
HANDS_ONLY_PREDS = None  # e.g., Path(OUT_ROOT) / "preds/test/effb0_hands_only_V1_test.csv"
FACE_HANDS_PREDS = None  # e.g., Path(OUT_ROOT) / "preds/test/effb0_face_hands_V1_test.csv"

# Auto-detect from OUT_ROOT if not specified
if FACE_HANDS_PREDS is None:
    candidates = list(Path(OUT_ROOT).glob("preds/test/*face_hands*.csv"))
    if candidates:
        FACE_HANDS_PREDS = sorted(candidates, key=lambda x: x.stat().st_mtime, reverse=True)[0]
        print(f"📁 Auto-detected face+hands predictions: {FACE_HANDS_PREDS.name}")

CLASS_NAMES = {
    0: "Safe driving",
    1: "Texting (R)",
    2: "Phone (R)",
    3: "Texting (L)",
    4: "Phone (L)",
    5: "Radio",
    6: "Drinking",
    7: "Reaching back",
    8: "Hair/makeup",
    9: "Passenger",
}

# ============================================================================
# HELPER FUNCTION
# ============================================================================
def load_and_compute_per_class_acc(pred_path, name):
    """Load predictions and compute per-class accuracy."""
    if pred_path is None or not Path(pred_path).exists():
        return None
    
    df = pd.read_csv(pred_path)
    
    # Extract true label from path
    def extract_class(path):
        parts = Path(path).parts
        for p in reversed(parts):
            if p.startswith("c") and len(p) == 2 and p[1].isdigit():
                return int(p[1])
        return -1
    
    df["true"] = df["path"].apply(extract_class)
    df["pred"] = df["pred_class_id"].apply(
        lambda x: int(x[1]) if isinstance(x, str) and x.startswith("c") else int(x)
    )
    
    # Per-class accuracy
    results = {}
    for c in range(10):
        class_df = df[df["true"] == c]
        if len(class_df) > 0:
            acc = (class_df["true"] == class_df["pred"]).mean()
            results[c] = {"accuracy": acc, "count": len(class_df)}
    
    return results

# ============================================================================
# LOAD ALL PREDICTIONS
# ============================================================================
print("📊 Loading predictions for comparison...\n")

modalities = {}
if FACE_ONLY_PREDS:
    modalities["Face Only"] = load_and_compute_per_class_acc(FACE_ONLY_PREDS, "face")
if HANDS_ONLY_PREDS:
    modalities["Hands Only"] = load_and_compute_per_class_acc(HANDS_ONLY_PREDS, "hands")
if FACE_HANDS_PREDS:
    modalities["Face+Hands"] = load_and_compute_per_class_acc(FACE_HANDS_PREDS, "face_hands")

if len(modalities) < 2:
    print("⚠️  Need at least 2 model variants to compare.")
    print("   Set FACE_ONLY_PREDS, HANDS_ONLY_PREDS, and/or FACE_HANDS_PREDS above.")
    print("\n   Currently available predictions:")
    for f in Path(OUT_ROOT).glob("preds/test/*.csv"):
        print(f"     - {f.name}")
else:
    # ============================================================================
    # CREATE COMPARISON TABLE
    # ============================================================================
    print("="*70)
    print("PER-CLASS ACCURACY COMPARISON")
    print("="*70)
    
    # Build comparison dataframe
    rows = []
    for c in range(10):
        row = {"Class": f"c{c}", "Name": CLASS_NAMES.get(c, f"Class {c}")}
        for mod_name, mod_data in modalities.items():
            if mod_data and c in mod_data:
                row[mod_name] = f"{mod_data[c]['accuracy']*100:.1f}%"
            else:
                row[mod_name] = "N/A"
        rows.append(row)
    
    comparison_df = pd.DataFrame(rows)
    print(comparison_df.to_string(index=False))
    
    # ============================================================================
    # DETERMINE MODALITY DOMINANCE
    # ============================================================================
    print("\n" + "="*70)
    print("MODALITY DOMINANCE ANALYSIS")
    print("="*70)
    
    if "Face Only" in modalities and "Hands Only" in modalities:
        for c in range(10):
            face_acc = modalities["Face Only"].get(c, {}).get("accuracy", 0) if modalities["Face Only"] else 0
            hands_acc = modalities["Hands Only"].get(c, {}).get("accuracy", 0) if modalities["Hands Only"] else 0
            
            diff = (hands_acc - face_acc) * 100
            if abs(diff) < 5:
                dominance = "⚖️  BOTH (similar)"
            elif diff > 0:
                dominance = f"🖐️ HANDS (+{diff:.1f}pp)"
            else:
                dominance = f"👤 FACE (+{-diff:.1f}pp)"
            
            print(f"  {CLASS_NAMES.get(c, f'c{c}'):20s}: {dominance}")
    
    # ============================================================================
    # VISUALIZATION
    # ============================================================================
    fig, ax = plt.subplots(figsize=(14, 6))
    
    x = np.arange(10)
    width = 0.25
    offset = 0
    
    colors = {"Face Only": "#FF6B6B", "Hands Only": "#4ECDC4", "Face+Hands": "#45B7D1"}
    
    for mod_name, mod_data in modalities.items():
        if mod_data:
            accs = [mod_data.get(c, {}).get("accuracy", 0) * 100 for c in range(10)]
            ax.bar(x + offset, accs, width, label=mod_name, color=colors.get(mod_name, "gray"))
            offset += width
    
    ax.set_xlabel("Class")
    ax.set_ylabel("Accuracy (%)")
    ax.set_title("Per-Class Accuracy: Face-Only vs Hands-Only vs Face+Hands")
    ax.set_xticks(x + width)
    ax.set_xticklabels([CLASS_NAMES.get(c, f"c{c}") for c in range(10)], rotation=45, ha="right")
    ax.legend()
    ax.set_ylim(0, 100)
    ax.axhline(y=50, color="gray", linestyle="--", alpha=0.5)
    plt.tight_layout()
    
    out_path = Path(OUT_ROOT) / "metrics" / "modality_comparison_per_class.png"
    out_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_path, dpi=150)
    plt.show()
    print(f"\n💾 Saved to {out_path}")


## 📊 11.4d Side-by-Side Confusion Matrix Comparison (Face vs Hands)

Compare confusion matrices for face-only vs hands-only models to see:
- **Which class pairs get confused** with each modality
- **Asymmetric confusions** (e.g., texting→phone but not phone→texting)
- **Modality-specific weaknesses** (hands model confuses drinking/reaching, face model confuses phone poses)

**Prerequisites:** Run predictions for both face-only and hands-only models.


In [ ]:
# 📊 Side-by-Side Confusion Matrix Comparison: Face vs Hands

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# ============================================================================
# CONFIGURATION - Set prediction file paths
# ============================================================================
FACE_ONLY_PREDS = None  # e.g., Path(OUT_ROOT) / "preds/test/effb0_face_only_V1_test.csv"
HANDS_ONLY_PREDS = None  # e.g., Path(OUT_ROOT) / "preds/test/effb0_hands_only_V1_test.csv"

# Try to auto-detect
for f in Path(OUT_ROOT).glob("preds/test/*.csv"):
    fname = f.name.lower()
    if "face_only" in fname and FACE_ONLY_PREDS is None:
        FACE_ONLY_PREDS = f
        print(f"📁 Auto-detected face-only: {f.name}")
    elif "hands_only" in fname and HANDS_ONLY_PREDS is None:
        HANDS_ONLY_PREDS = f
        print(f"📁 Auto-detected hands-only: {f.name}")

CLASS_NAMES_SHORT = ["Safe", "TxtR", "PhR", "TxtL", "PhL", "Radio", "Drink", "Reach", "Hair", "Pass"]

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================
def load_predictions_with_labels(pred_path):
    """Load predictions and extract true labels from paths."""
    df = pd.read_csv(pred_path)
    
    def extract_class(path):
        parts = Path(path).parts
        for p in reversed(parts):
            if p.startswith("c") and len(p) == 2 and p[1].isdigit():
                return int(p[1])
        return -1
    
    df["true"] = df["path"].apply(extract_class)
    df["pred"] = df["pred_class_id"].apply(
        lambda x: int(x[1]) if isinstance(x, str) and x.startswith("c") else int(x)
    )
    return df[df["true"] >= 0]

def compute_confusion_diff(cm1, cm2):
    """Compute difference matrix: positive = cm1 is worse (more confusions)."""
    # Normalize by row (per-class)
    cm1_norm = cm1.astype(float) / cm1.sum(axis=1, keepdims=True)
    cm2_norm = cm2.astype(float) / cm2.sum(axis=1, keepdims=True)
    # Replace NaN with 0
    cm1_norm = np.nan_to_num(cm1_norm)
    cm2_norm = np.nan_to_num(cm2_norm)
    return cm1_norm - cm2_norm

# ============================================================================
# LOAD AND COMPUTE
# ============================================================================
if FACE_ONLY_PREDS is None or HANDS_ONLY_PREDS is None:
    print("⚠️  Need both face-only AND hands-only prediction files.")
    print("\n   Set FACE_ONLY_PREDS and HANDS_ONLY_PREDS above, or train the models first.")
    print("\n   Currently available predictions:")
    for f in Path(OUT_ROOT).glob("preds/test/*.csv"):
        print(f"     - {f.name}")
else:
    print("📊 Loading predictions...\n")
    
    face_df = load_predictions_with_labels(FACE_ONLY_PREDS)
    hands_df = load_predictions_with_labels(HANDS_ONLY_PREDS)
    
    # Compute confusion matrices
    cm_face = confusion_matrix(face_df["true"], face_df["pred"], labels=range(10))
    cm_hands = confusion_matrix(hands_df["true"], hands_df["pred"], labels=range(10))
    
    # Accuracy
    acc_face = (face_df["true"] == face_df["pred"]).mean()
    acc_hands = (hands_df["true"] == hands_df["pred"]).mean()
    
    # ============================================================================
    # SIDE-BY-SIDE VISUALIZATION
    # ============================================================================
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    # Face confusion matrix
    sns.heatmap(cm_face, annot=True, fmt="d", cmap="Blues", ax=axes[0],
                xticklabels=CLASS_NAMES_SHORT, yticklabels=CLASS_NAMES_SHORT)
    axes[0].set_title(f"Face-Only Model\nAccuracy: {acc_face*100:.1f}%", fontsize=12)
    axes[0].set_xlabel("Predicted")
    axes[0].set_ylabel("True")
    
    # Hands confusion matrix
    sns.heatmap(cm_hands, annot=True, fmt="d", cmap="Greens", ax=axes[1],
                xticklabels=CLASS_NAMES_SHORT, yticklabels=CLASS_NAMES_SHORT)
    axes[1].set_title(f"Hands-Only Model\nAccuracy: {acc_hands*100:.1f}%", fontsize=12)
    axes[1].set_xlabel("Predicted")
    axes[1].set_ylabel("True")
    
    # Difference matrix (Face - Hands): Red = Face is worse, Blue = Hands is worse
    diff = compute_confusion_diff(cm_face, cm_hands)
    sns.heatmap(diff * 100, annot=True, fmt=".1f", cmap="RdBu_r", center=0, ax=axes[2],
                xticklabels=CLASS_NAMES_SHORT, yticklabels=CLASS_NAMES_SHORT,
                vmin=-20, vmax=20)
    axes[2].set_title("Difference (Face − Hands)\nRed = Face worse, Blue = Hands worse", fontsize=12)
    axes[2].set_xlabel("Predicted")
    axes[2].set_ylabel("True")
    
    plt.tight_layout()
    
    out_path = Path(OUT_ROOT) / "metrics" / "confusion_face_vs_hands_comparison.png"
    out_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_path, dpi=150)
    plt.show()
    print(f"\n💾 Saved to {out_path}")
    
    # ============================================================================
    # TOP CONFUSION PAIRS ANALYSIS
    # ============================================================================
    print("\n" + "="*70)
    print("🔍 TOP CONFUSION PAIRS BY MODALITY")
    print("="*70)
    
    CLASS_NAMES_FULL = {
        0: "Safe driving", 1: "Texting (R)", 2: "Phone (R)", 3: "Texting (L)",
        4: "Phone (L)", 5: "Radio", 6: "Drinking", 7: "Reaching back",
        8: "Hair/makeup", 9: "Passenger"
    }
    
    def get_top_confusions(cm, top_n=5):
        """Get top N off-diagonal confusions."""
        confusions = []
        for i in range(10):
            for j in range(10):
                if i != j and cm[i, j] > 0:
                    confusions.append((i, j, cm[i, j]))
        return sorted(confusions, key=lambda x: -x[2])[:top_n]
    
    print("\n👤 FACE-ONLY Top Confusions:")
    for true_c, pred_c, count in get_top_confusions(cm_face):
        print(f"   {CLASS_NAMES_FULL[true_c]:20s} → {CLASS_NAMES_FULL[pred_c]:20s}: {count} errors")
    
    print("\n🖐️ HANDS-ONLY Top Confusions:")
    for true_c, pred_c, count in get_top_confusions(cm_hands):
        print(f"   {CLASS_NAMES_FULL[true_c]:20s} → {CLASS_NAMES_FULL[pred_c]:20s}: {count} errors")
    
    # ============================================================================
    # MODALITY-SPECIFIC INSIGHTS
    # ============================================================================
    print("\n" + "="*70)
    print("💡 MODALITY-SPECIFIC INSIGHTS")
    print("="*70)
    
    # Find pairs where one modality is much better
    for i in range(10):
        for j in range(10):
            if i != j:
                face_rate = cm_face[i, j] / cm_face[i].sum() if cm_face[i].sum() > 0 else 0
                hands_rate = cm_hands[i, j] / cm_hands[i].sum() if cm_hands[i].sum() > 0 else 0
                
                diff_pct = (face_rate - hands_rate) * 100
                if abs(diff_pct) > 5:  # 5% difference threshold
                    if diff_pct > 0:
                        print(f"   🖐️ Hands better at: {CLASS_NAMES_FULL[i][:15]:15s} vs {CLASS_NAMES_FULL[j][:15]:15s} (+{diff_pct:.1f}pp)")
                    else:
                        print(f"   👤 Face better at:  {CLASS_NAMES_FULL[i][:15]:15s} vs {CLASS_NAMES_FULL[j][:15]:15s} (+{-diff_pct:.1f}pp)")


## 🔥 11.5 Grad-CAM Visualizations

**Grad-CAM (Gradient-weighted Class Activation Mapping)** shows which regions of the image the model focuses on when making predictions.

**Use cases:**
- **Full image models:** Verify the model looks at face/hands, not background
- **Hybrid crop models:** See which specific features (hand position, facial expression) matter most
- **Thesis comparison:** Visual evidence of WHY ROI cropping helps

This cell generates:
1. Grad-CAM heatmaps for sample images
2. Comparison of correct vs misclassified predictions
3. Per-class attention patterns
4. Saved visualizations for your thesis


In [ ]:
# 🔥 Grad-CAM Visualization
!pip -q install grad-cam

import json
import random
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision import transforms

# ===== CONFIGURATION =====
# Derive tag from checkpoint that was already selected in cell 11.2
if "LATEST_CKPT" in dir() and LATEST_CKPT:
    GRADCAM_TAG = Path(LATEST_CKPT).parent.parent.name  # e.g., effb0_InsightFace_MP_face_hands_V3
else:
    GRADCAM_TAG = globals().get("TRAIN_TAG") or "effb0_hybrid_face_hands"

MODEL_NAME = globals().get("MODEL_NAME") or "efficientnet_b0"  # efficientnet_b0, resnet18, convnext_tiny
# Use the same split as predictions (PRED_SPLIT), or default to "test"
SPLIT_TO_ANALYZE = globals().get("PRED_SPLIT") or "test"
N_SAMPLES_PER_CLASS = 3  # How many samples per class to visualize
N_MISCLASSIFIED = 6  # How many misclassified examples to show
IMAGE_SIZE = 224

# ===== FIND CHECKPOINT =====
# Use LATEST_CKPT if already selected (from cell 11.2), otherwise search
if "LATEST_CKPT" in dir() and LATEST_CKPT and Path(LATEST_CKPT).exists():
    ckpt_path = Path(LATEST_CKPT)
    print(f"📁 Using pre-selected checkpoint: {ckpt_path}")
else:
    run_base = Path(CKPT_ROOT) / "runs" / GRADCAM_TAG
    all_runs = sorted(run_base.glob("*/"))
    if not all_runs:
        raise FileNotFoundError(f"No run folders found under {run_base}")
    latest_run = all_runs[-1]
    ckpt_path = latest_run / "best.pt"
    if not ckpt_path.exists():
        ckpt_path = latest_run / "last.pt"
    print(f"📁 Using checkpoint: {ckpt_path}")

# ===== LOAD MODEL =====
from ddriver.models.registry import build_model, register_timm_backbone

# Register timm backbone if needed
try:
    register_timm_backbone(MODEL_NAME)
except (ImportError, ValueError):
    pass  # Already registered or custom model

# Load checkpoint to get num_classes
ckpt = torch.load(ckpt_path, map_location="cpu")
num_classes = ckpt["model_state_dict"]["classifier.weight"].shape[0] if "classifier.weight" in ckpt["model_state_dict"] else 10

model = build_model(MODEL_NAME, num_classes=num_classes, pretrained=False)
model.load_state_dict(ckpt["model_state_dict"])
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"✅ Loaded model: {MODEL_NAME} with {num_classes} classes")

# ===== DETERMINE TARGET LAYER FOR GRAD-CAM =====
# Different architectures have different final conv layers
# Handle both wrapped (model.backbone) and unwrapped (direct timm) models
def get_target_layers(model, model_name):
    """Get appropriate target layers for Grad-CAM based on model architecture."""
    # Check if model has backbone wrapper
    base = getattr(model, "backbone", model)
    
    if "efficientnet" in model_name.lower():
        # timm EfficientNet: use conv_head or last block
        if hasattr(base, "conv_head"):
            return [base.conv_head]
        elif hasattr(base, "blocks"):
            return [base.blocks[-1]]
        elif hasattr(base, "features"):
            return [base.features[-1]]
    elif "resnet" in model_name.lower():
        if hasattr(base, "layer4"):
            return [base.layer4[-1]]
    elif "convnext" in model_name.lower():
        if hasattr(base, "stages"):
            return [base.stages[-1]]
        elif hasattr(base, "features"):
            return [base.features[-1]]
    
    # Fallback: try common patterns
    for attr in ["features", "blocks", "stages", "layer4"]:
        if hasattr(base, attr):
            layer = getattr(base, attr)
            if hasattr(layer, "__getitem__"):
                return [layer[-1]]
    
    # Last resort: get second-to-last child
    children = list(base.children())
    if len(children) >= 2:
        return [children[-2]]
    raise ValueError(f"Could not determine target layer for {model_name}")

target_layers = get_target_layers(model, MODEL_NAME)
print(f"🎯 Target layer for Grad-CAM: {target_layers[0].__class__.__name__}")

# ===== LOAD PREDICTIONS AND DATA =====
# Find the predictions CSV - use PRED_TAG if set, otherwise construct from GRADCAM_TAG
# Predictions are saved as {RUN_TAG}_{split}.csv, so we need to match that
if "PRED_TAG" in dir() and PRED_TAG:
    preds_tag = PRED_TAG
else:
    # Construct the expected predictions tag: {GRADCAM_TAG}_{split}
    preds_tag = f"{GRADCAM_TAG}_{SPLIT_TO_ANALYZE}"

preds_csv = Path(OUT_ROOT) / "preds" / SPLIT_TO_ANALYZE / f"{preds_tag}.csv"
if not preds_csv.exists():
    # Also try without the split suffix (in case predictions were saved differently)
    alt_preds_csv = Path(OUT_ROOT) / "preds" / SPLIT_TO_ANALYZE / f"{GRADCAM_TAG}.csv"
    if alt_preds_csv.exists():
        preds_csv = alt_preds_csv
    else:
        raise FileNotFoundError(
            f"Predictions not found at:\n  - {preds_csv}\n  - {alt_preds_csv}\n"
            f"Run the prediction cell first!"
        )

preds_df = pd.read_csv(preds_csv)
print(f"📊 Loaded {len(preds_df)} predictions from {preds_csv}")

# Determine data root and manifest path based on data source
if "USE_HYBRID" in globals() and USE_HYBRID:
    data_root = Path(os.environ.get("HYBRID_ROOT_LOCAL", Path(OUT_ROOT) / "hybrid"))
    roi_variant = globals().get("ROI_VARIANT", "face_hands")
    manifest_path = data_root / f"manifest_{roi_variant}.csv"
elif "USE_YOLO" in globals() and USE_YOLO:
    data_root = Path(os.environ.get("YOLO_ROOT_LOCAL", Path(OUT_ROOT) / "yolo"))
    roi_variant = globals().get("ROI_VARIANT", "face_hands")
    manifest_path = data_root / f"manifest_{roi_variant}.csv"
elif "USE_MEDIAPIPE" in globals() and USE_MEDIAPIPE:
    data_root = Path(OUT_ROOT) / "mediapipe"
    roi_variant = globals().get("ROI_VARIANT", "face_hands")
    manifest_path = data_root / f"manifest_{roi_variant}.csv"
else:
    data_root = Path(DATASET_ROOT)
    manifest_path = Path(OUT_ROOT) / "manifests" / "manifest.csv"
print(f"📁 Data root: {data_root}")

# Helper to find image with multiple fallback paths
def find_image_path(path_str, data_root, dataset_root, roi_variant=None):
    """
    Try multiple locations to find the image:
    1. If path is absolute and exists, use it
    2. Try data_root / path (for hybrid/yolo/mediapipe)
    3. Try data_root / variant / path (if variant subfolder)
    4. Try dataset_root / path (original images - warns if used)
    5. Extract relative path and try common patterns
    """
    path = Path(path_str)
    
    # 1. Absolute path that exists
    if path.is_absolute() and path.exists():
        return path, False  # (path, is_fallback)
    
    # 2. Try data_root / path directly
    candidate = data_root / path_str
    if candidate.exists():
        return candidate, False
    
    # 3. If path doesn't start with variant, try adding it
    if roi_variant and not path_str.startswith(roi_variant):
        candidate = data_root / roi_variant / path_str
        if candidate.exists():
            return candidate, False
    
    # 4. Try with just the filename in the expected structure
    # Extract class folder and filename from path
    parts = Path(path_str).parts
    if len(parts) >= 2:
        class_folder = [p for p in parts if p.startswith('c') and len(p) == 2]
        if class_folder:
            filename = Path(path_str).name
            # Try: data_root / variant / class / filename
            if roi_variant:
                candidate = data_root / roi_variant / class_folder[0] / filename
                if candidate.exists():
                    return candidate, False
    
    # 5. Fallback to original dataset (this shows WRONG images for cropped models!)
    candidate = Path(dataset_root) / path_str
    if candidate.exists():
        return candidate, True  # Mark as fallback
    
    # Try removing any prefix from the path to get relative path
    for marker in ["auc.distracted.driver", "train/", "test/"]:
        if marker in path_str:
            idx = path_str.find(marker)
            rel_path = path_str[idx:]
            candidate = Path(dataset_root) / rel_path
            if candidate.exists():
                return candidate, True
    
    return None, False

# Track fallback warnings (use list to allow mutation in nested scope)
_fallback_state = {"warned": False}

# Load manifest to get ground truth labels
manifest_df = pd.read_csv(manifest_path)
print(f"📋 Loaded manifest with {len(manifest_df)} entries from {manifest_path}")

# Debug: Show sample paths to help diagnose path issues
print(f"📍 Sample manifest paths: {manifest_df['path'].head(2).tolist()}")

# Merge predictions with manifest to get ground truth labels
# Predictions have: path (may be absolute), pred_class_id (e.g., "c0")
# Manifest has: path (relative crop path), class_id (e.g., "c0")
# Match on CLASS + FILENAME to avoid duplicates (same filename exists in c0/, c1/, etc.)

# Show data mode info (path format doesn't indicate what was actually loaded)
if "USE_HYBRID" in globals() and USE_HYBRID:
    print("✅ HYBRID mode - model was trained/predicted on face+hands crops")
elif "USE_YOLO" in globals() and USE_YOLO:
    print("✅ YOLO mode - model was trained/predicted on YOLO-detected crops")
elif "USE_MEDIAPIPE" in globals() and USE_MEDIAPIPE:
    print("✅ MEDIAPIPE mode - model was trained/predicted on MediaPipe crops")
else:
    print("📷 FULL IMAGE mode - model was trained/predicted on original full images")

# Extract class folder (c0-c9) from prediction path
def extract_class_from_path(p):
    parts = Path(p).parts
    for part in parts:
        if part.startswith('c') and len(part) == 2 and part[1].isdigit():
            return part
    return ""

preds_df["_class"] = preds_df["path"].apply(extract_class_from_path)
preds_df["_filename"] = preds_df["path"].apply(lambda p: Path(p).name)

# Manifest has class_id column (c0-c9)
manifest_df["_class"] = manifest_df["class_id"]
manifest_df["_filename"] = manifest_df["path"].apply(lambda p: Path(p).name)

# Merge on BOTH class and filename to get unique matches
preds_df = preds_df.merge(
    manifest_df[["_class", "_filename", "path", "class_id"]].rename(columns={"path": "crop_path"}),
    on=["_class", "_filename"],
    how="left"
)

# Check for unmatched or duplicate rows
n_original = len(preds_df)
n_unmatched = preds_df["class_id"].isna().sum()
if n_unmatched > 0:
    print(f"⚠️  {n_unmatched}/{n_original} predictions couldn't be matched to manifest")
    preds_df = preds_df.dropna(subset=["class_id"])
print(f"✅ Final merged dataset: {len(preds_df)} rows (from {n_original} after merge)")

# Use crop_path for visualization (points to actual cropped images)
preds_df["vis_path"] = preds_df["crop_path"]

# Convert class_id strings (c0-c9) to integers (0-9)
def class_to_int(class_id):
    if pd.isna(class_id):
        return -1
    if isinstance(class_id, str) and class_id.startswith("c"):
        return int(class_id[1:])
    return int(class_id)

preds_df["label"] = preds_df["class_id"].apply(class_to_int)
preds_df["pred"] = preds_df["pred_class_id"].apply(class_to_int)

# Add placeholder confidence (not available from current prediction code)
if "confidence" not in preds_df.columns:
    preds_df["confidence"] = 1.0  # Placeholder

print(f"✅ Merged predictions with labels. Columns: {list(preds_df.columns)}")
print(f"📍 Will visualize using crop_path: {preds_df['vis_path'].head(2).tolist()}")

# ===== CLASS NAMES =====
CLASS_NAMES = {
    0: "Safe driving",
    1: "Texting (right)",
    2: "Talking (right)",
    3: "Texting (left)",
    4: "Talking (left)",
    5: "Operating radio",
    6: "Drinking",
    7: "Reaching behind",
    8: "Hair/makeup",
    9: "Talking to passenger",
}

# ===== TRANSFORMS =====
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_image(path):
    """Load image and return both tensor and RGB numpy array."""
    img = Image.open(path).convert("RGB")
    img_resized = img.resize((IMAGE_SIZE, IMAGE_SIZE))
    img_np = np.array(img_resized) / 255.0  # Normalized 0-1 for overlay
    img_tensor = transform(img).unsqueeze(0)
    return img_tensor, img_np

# ===== GENERATE GRAD-CAM =====
cam = GradCAM(model=model, target_layers=target_layers)

def generate_gradcam(img_path, target_class=None):
    """Generate Grad-CAM visualization for an image."""
    img_tensor, img_np = load_image(img_path)
    img_tensor = img_tensor.to(device)
    
    # Generate CAM (None = use predicted class)
    grayscale_cam = cam(input_tensor=img_tensor, targets=None)
    grayscale_cam = grayscale_cam[0, :]
    
    # Overlay on image
    visualization = show_cam_on_image(img_np.astype(np.float32), grayscale_cam, use_rgb=True)
    return visualization, grayscale_cam

# ===== 1. SAMPLE CORRECT PREDICTIONS PER CLASS =====
print("\n🎨 Generating Grad-CAM for correct predictions per class...")

correct_df = preds_df[preds_df["label"] == preds_df["pred"]]
fig, axes = plt.subplots(num_classes, N_SAMPLES_PER_CLASS * 2, figsize=(N_SAMPLES_PER_CLASS * 6, num_classes * 3))

for class_id in range(num_classes):
    class_samples = correct_df[correct_df["label"] == class_id]
    samples = class_samples.sample(min(N_SAMPLES_PER_CLASS, len(class_samples)), random_state=42)
    
    for i, (_, row) in enumerate(samples.iterrows()):
        if i >= N_SAMPLES_PER_CLASS:
            break
        
        # Use vis_path (crop path from manifest) for visualization
        vis_path_str = row.get("vis_path", row["path"])
        roi_var = globals().get("ROI_VARIANT") if ("USE_HYBRID" in globals() or "USE_YOLO" in globals() or "USE_MEDIAPIPE" in globals()) else None
        img_path, is_fallback = find_image_path(vis_path_str, data_root, DATASET_ROOT, roi_var)
        
        if is_fallback and not _fallback_state["warned"]:
            print(f"⚠️  WARNING: Using ORIGINAL full images for visualization (cropped images not found)")
            print(f"   This means Grad-CAM heatmaps are overlaid on wrong images!")
            print(f"   Expected path: {data_root / vis_path_str}")
            _fallback_state["warned"] = True
        
        if img_path and img_path.exists():
            # Original image
            img = Image.open(img_path).convert("RGB").resize((IMAGE_SIZE, IMAGE_SIZE))
            axes[class_id, i * 2].imshow(img)
            axes[class_id, i * 2].set_title(f"{CLASS_NAMES.get(class_id, f'Class {class_id}')}", fontsize=8)
            axes[class_id, i * 2].axis("off")
            
            # Grad-CAM
            viz, _ = generate_gradcam(img_path)
            axes[class_id, i * 2 + 1].imshow(viz)
            axes[class_id, i * 2 + 1].set_title(f"Grad-CAM (conf: {row['confidence']:.2f})", fontsize=8)
            axes[class_id, i * 2 + 1].axis("off")
        else:
            axes[class_id, i * 2].text(0.5, 0.5, "Not found", ha="center", va="center")
            axes[class_id, i * 2].axis("off")
            axes[class_id, i * 2 + 1].axis("off")

plt.suptitle(f"Grad-CAM: Correct Predictions per Class ({MODEL_NAME})", fontsize=14, y=1.02)
plt.tight_layout()

# Save
gradcam_dir = Path(OUT_ROOT) / "gradcam" / GRADCAM_TAG
gradcam_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(gradcam_dir / "correct_per_class.png", dpi=150, bbox_inches="tight")
plt.show()
print(f"💾 Saved to {gradcam_dir / 'correct_per_class.png'}")

# ===== 2. MISCLASSIFIED EXAMPLES =====
print("\n🔴 Generating Grad-CAM for misclassified examples...")

misclassified_df = preds_df[preds_df["label"] != preds_df["pred"]]
if len(misclassified_df) > 0:
    samples = misclassified_df.sample(min(N_MISCLASSIFIED, len(misclassified_df)), random_state=42)
    
    fig, axes = plt.subplots(len(samples), 3, figsize=(12, len(samples) * 3))
    if len(samples) == 1:
        axes = axes.reshape(1, -1)
    
    for i, (_, row) in enumerate(samples.iterrows()):
        # Use vis_path (crop path from manifest) for visualization
        vis_path_str = row.get("vis_path", row["path"])
        roi_var = globals().get("ROI_VARIANT") if ("USE_HYBRID" in globals() or "USE_YOLO" in globals() or "USE_MEDIAPIPE" in globals()) else None
        img_path, is_fallback = find_image_path(vis_path_str, data_root, DATASET_ROOT, roi_var)
        
        if img_path and img_path.exists():
            # Original
            img = Image.open(img_path).convert("RGB").resize((IMAGE_SIZE, IMAGE_SIZE))
            axes[i, 0].imshow(img)
            axes[i, 0].set_title(f"True: {CLASS_NAMES.get(int(row['label']), row['label'])}", fontsize=9)
            axes[i, 0].axis("off")
            
            # Grad-CAM
            viz, heatmap = generate_gradcam(img_path)
            axes[i, 1].imshow(viz)
            axes[i, 1].set_title(f"Pred: {CLASS_NAMES.get(int(row['pred']), row['pred'])} ({row['confidence']:.2f})", fontsize=9)
            axes[i, 1].axis("off")
            
            # Heatmap only
            axes[i, 2].imshow(heatmap, cmap="jet")
            axes[i, 2].set_title("Attention heatmap", fontsize=9)
            axes[i, 2].axis("off")
    
    plt.suptitle(f"Grad-CAM: Misclassified Examples ({MODEL_NAME})", fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig(gradcam_dir / "misclassified.png", dpi=150, bbox_inches="tight")
    plt.show()
    print(f"💾 Saved to {gradcam_dir / 'misclassified.png'}")
else:
    print("✅ No misclassified examples found!")

# ===== 3. SUMMARY STATISTICS =====
print("\n📊 Grad-CAM Analysis Summary:")
print(f"   Model: {MODEL_NAME}")
print(f"   Checkpoint: {ckpt_path.name}")
print(f"   Split analyzed: {SPLIT_TO_ANALYZE}")
print(f"   Total predictions: {len(preds_df)}")
print(f"   Correct: {len(correct_df)} ({100*len(correct_df)/len(preds_df):.1f}%)")
print(f"   Misclassified: {len(misclassified_df)} ({100*len(misclassified_df)/len(preds_df):.1f}%)")
print(f"\n📁 Visualizations saved to: {gradcam_dir}")


## 🎯 11.5b Advanced Grad-CAM Analysis (Stratified Sampling)

This cell generates a **systematic Grad-CAM analysis** using stratified sampling:

**Four categories of predictions:**
1. ✅ **Correct + High confidence** - What the model considers strong evidence
2. 🤔 **Correct + Low confidence** - Borderline cases, "lucky guesses"
3. ❌ **Wrong + High confidence** - **Most important!** Reveals shortcuts/spurious patterns
4. 😕 **Wrong + Low confidence** - Reasonable confusions, ambiguous cases

**Outputs:**
- **Thesis gallery** (12-24 images): Representative examples for each "story"
- **Extended gallery** (100-200 images): Full review set for appendix

**Key questions this answers:**
- Does the model focus on the right evidence when correct?
- When wrong, is it looking at shortcuts (background/identity cues) or genuinely ambiguous actions?
- Which classes rely on face vs hands cues?


In [ ]:
# 🎯 Advanced Grad-CAM Analysis with Stratified Sampling
# Requires: Run cell 11.5 first to set up model, predictions, and Grad-CAM

import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# ============================================================================
# CONFIGURATION
# ============================================================================
N_SAMPLES_PER_CATEGORY = 5       # Per class, per category (correct/wrong × high/low conf)
CONFIDENCE_THRESHOLD = 0.8       # Above = high conf, below = low conf
THESIS_SAMPLES_PER_STORY = 3     # For the main thesis gallery
SAVE_EXTENDED_GALLERY = True     # Save all samples to appendix folder

# Reuse variables from cell 11.5
# Assumes: preds_df, model, cam, data_root, CLASS_NAMES, generate_gradcam, find_image_path

# ============================================================================
# STRATIFIED SAMPLING
# ============================================================================
print("📊 Stratifying predictions by correctness and confidence...\n")

# Ensure we have confidence column
if "confidence" not in preds_df.columns:
    print("⚠️  No 'confidence' column in predictions. Using placeholder (1.0).")
    print("   Re-run predictions with latest code to get real confidence scores.")
    preds_df["confidence"] = 1.0

# Create category columns
preds_df["correct"] = preds_df["label"] == preds_df["pred"]
preds_df["high_conf"] = preds_df["confidence"] >= CONFIDENCE_THRESHOLD

# Define the 4 categories
categories = {
    "correct_high": preds_df[preds_df["correct"] & preds_df["high_conf"]],
    "correct_low": preds_df[preds_df["correct"] & ~preds_df["high_conf"]],
    "wrong_high": preds_df[~preds_df["correct"] & preds_df["high_conf"]],
    "wrong_low": preds_df[~preds_df["correct"] & ~preds_df["high_conf"]],
}

print("Category breakdown:")
for cat_name, cat_df in categories.items():
    print(f"  {cat_name}: {len(cat_df)} samples ({100*len(cat_df)/len(preds_df):.1f}%)")

# ============================================================================
# SAMPLE SELECTION
# ============================================================================
print("\n🎲 Selecting stratified samples...")

selected_samples = {cat: [] for cat in categories}

for cat_name, cat_df in categories.items():
    if len(cat_df) == 0:
        continue
    
    for class_id in range(10):
        class_df = cat_df[cat_df["label"] == class_id]
        if len(class_df) == 0:
            continue
        
        # Sort by confidence (descending for high_conf categories, ascending for low)
        if "high" in cat_name:
            class_df = class_df.sort_values("confidence", ascending=False)
        else:
            class_df = class_df.sort_values("confidence", ascending=True)
        
        # Take top N samples
        samples = class_df.head(N_SAMPLES_PER_CATEGORY)
        selected_samples[cat_name].extend(samples.to_dict("records"))

print("\nSelected samples per category:")
for cat_name, samples in selected_samples.items():
    print(f"  {cat_name}: {len(samples)} samples")

# ============================================================================
# GENERATE GRAD-CAM FOR ALL SELECTED SAMPLES
# ============================================================================
print("\n🔥 Generating Grad-CAM visualizations...")

gradcam_results = {cat: [] for cat in categories}
roi_var = globals().get("ROI_VARIANT") if any(globals().get(f"USE_{m}") for m in ["HYBRID", "YOLO", "MEDIAPIPE"]) else None

for cat_name, samples in selected_samples.items():
    print(f"\n  Processing {cat_name}...")
    for sample in samples:
        vis_path = sample.get("vis_path", sample.get("crop_path", sample["path"]))
        img_path, _ = find_image_path(vis_path, data_root, DATASET_ROOT, roi_var)
        
        if img_path and img_path.exists():
            try:
                viz, heatmap = generate_gradcam(img_path)
                gradcam_results[cat_name].append({
                    **sample,
                    "img_path": str(img_path),
                    "gradcam_viz": viz,
                    "heatmap": heatmap,
                })
            except Exception as e:
                print(f"    ⚠️ Failed for {vis_path}: {e}")

print("\n✅ Grad-CAM generation complete!")
for cat_name, results in gradcam_results.items():
    print(f"  {cat_name}: {len(results)} visualizations")

# ============================================================================
# SAVE EXTENDED GALLERY (for appendix)
# ============================================================================
if SAVE_EXTENDED_GALLERY:
    gallery_dir = Path(OUT_ROOT) / "gradcam" / GRADCAM_TAG / "stratified_gallery"
    gallery_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\n💾 Saving extended gallery to {gallery_dir}...")
    
    for cat_name, results in gradcam_results.items():
        cat_dir = gallery_dir / cat_name
        cat_dir.mkdir(exist_ok=True)
        
        for i, r in enumerate(results):
            true_label = CLASS_NAMES.get(int(r["label"]), f"c{r['label']}")
            pred_label = CLASS_NAMES.get(int(r["pred"]), f"c{r['pred']}")
            conf = r["confidence"]
            
            # Save visualization
            fig, axes = plt.subplots(1, 3, figsize=(12, 4))
            
            # Original image
            img = Image.open(r["img_path"]).convert("RGB").resize((IMAGE_SIZE, IMAGE_SIZE))
            axes[0].imshow(img)
            axes[0].set_title(f"True: {true_label}", fontsize=10)
            axes[0].axis("off")
            
            # Grad-CAM overlay
            axes[1].imshow(r["gradcam_viz"])
            axes[1].set_title(f"Pred: {pred_label} ({conf:.2f})", fontsize=10)
            axes[1].axis("off")
            
            # Heatmap
            axes[2].imshow(r["heatmap"], cmap="jet")
            axes[2].set_title("Attention", fontsize=10)
            axes[2].axis("off")
            
            plt.tight_layout()
            plt.savefig(cat_dir / f"{i:03d}_true{r['label']}_pred{r['pred']}_conf{conf:.2f}.png", dpi=100)
            plt.close()
    
    print(f"✅ Extended gallery saved!")

# ============================================================================
# CREATE THESIS-READY SUMMARY FIGURE
# ============================================================================
print("\n📝 Creating thesis-ready summary figure...")

# Focus on the most interesting category: WRONG + HIGH CONFIDENCE
wrong_high = gradcam_results.get("wrong_high", [])[:12]  # Top 12 most confident errors

if len(wrong_high) > 0:
    n_cols = 4
    n_rows = min(3, (len(wrong_high) + n_cols - 1) // n_cols)
    fig, axes = plt.subplots(n_rows, n_cols * 2, figsize=(n_cols * 6, n_rows * 3))
    axes = axes.reshape(-1) if n_rows > 1 else axes
    
    for i, r in enumerate(wrong_high[:n_rows * n_cols]):
        true_label = CLASS_NAMES.get(int(r["label"]), f"c{r['label']}")
        pred_label = CLASS_NAMES.get(int(r["pred"]), f"c{r['pred']}")
        
        # Original
        img = Image.open(r["img_path"]).convert("RGB").resize((IMAGE_SIZE, IMAGE_SIZE))
        axes[i * 2].imshow(img)
        axes[i * 2].set_title(f"True: {true_label}", fontsize=8)
        axes[i * 2].axis("off")
        
        # Grad-CAM
        axes[i * 2 + 1].imshow(r["gradcam_viz"])
        axes[i * 2 + 1].set_title(f"Pred: {pred_label} ({r['confidence']:.2f})", fontsize=8)
        axes[i * 2 + 1].axis("off")
    
    # Hide unused axes
    for j in range(len(wrong_high) * 2, len(axes)):
        axes[j].axis("off")
    
    plt.suptitle("High-Confidence Errors: Where Does the Model Look When Wrong?", fontsize=12, y=1.02)
    plt.tight_layout()
    
    thesis_path = Path(OUT_ROOT) / "gradcam" / GRADCAM_TAG / "thesis_high_conf_errors.png"
    plt.savefig(thesis_path, dpi=150, bbox_inches="tight")
    plt.show()
    print(f"💾 Thesis figure saved to {thesis_path}")
else:
    print("ℹ️  No high-confidence errors found (model may be very accurate or conservative)")

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================
print("\n" + "="*60)
print("📊 STRATIFIED GRAD-CAM ANALYSIS SUMMARY")
print("="*60)
print(f"\nModel: {MODEL_NAME}")
print(f"Checkpoint: {Path(LATEST_CKPT).name if 'LATEST_CKPT' in dir() else 'unknown'}")
print(f"Confidence threshold: {CONFIDENCE_THRESHOLD}")
print(f"\nCategory distribution:")
for cat_name, cat_df in categories.items():
    emoji = {"correct_high": "✅🔥", "correct_low": "✅🤔", "wrong_high": "❌🔥", "wrong_low": "❌🤔"}[cat_name]
    print(f"  {emoji} {cat_name}: {len(cat_df)} ({100*len(cat_df)/len(preds_df):.1f}%)")

print(f"\n📁 Outputs saved to: {Path(OUT_ROOT) / 'gradcam' / GRADCAM_TAG}")
print(f"   - stratified_gallery/  (extended gallery for appendix)")
print(f"   - thesis_high_conf_errors.png  (main thesis figure)")


## 🔍 11.5c Grad-CAM for Specific Error Types

Investigate **specific confusion pairs** to understand why the model fails.

Example pairs to analyze:
- **Texting Left → Talking Left**: Same side, different action (phone vs hand position)
- **Drinking → Reaching Back**: Similar arm motion
- **Texting Right → Phone Right**: Both involve phone, different grip
- **Safe → Texting**: Why does the model hallucinate phone use?

This cell generates Grad-CAM visualizations for all examples of a specific (true→pred) confusion pair.


In [ ]:
# 🔍 Grad-CAM for Specific Error Types
# Requires: Run cell 11.5 first to set up model, predictions, and Grad-CAM

import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# ============================================================================
# CONFIGURATION - Which confusion pair to investigate?
# ============================================================================
# Set the TRUE class and PREDICTED class to investigate
TRUE_CLASS = 3   # Texting Left
PRED_CLASS = 4   # Phone Left

# You can also specify multiple pairs to analyze at once:
CONFUSION_PAIRS = [
    (3, 4, "Texting Left → Phone Left"),      # Similar: both left side, phone involved
    (1, 2, "Texting Right → Phone Right"),    # Similar: both right side, phone
    (6, 7, "Drinking → Reaching Back"),       # Similar: arm raised
    (0, 1, "Safe → Texting Right"),           # Why hallucinate phone?
    (9, 0, "Passenger → Safe"),               # Head position confusion
]

MAX_SAMPLES = 8  # Maximum samples per confusion pair

CLASS_NAMES = {
    0: "Safe driving", 1: "Texting (R)", 2: "Phone (R)", 3: "Texting (L)",
    4: "Phone (L)", 5: "Radio", 6: "Drinking", 7: "Reaching back",
    8: "Hair/makeup", 9: "Passenger"
}

# ============================================================================
# CHECK PREREQUISITES
# ============================================================================
required_vars = ["preds_df", "model", "cam", "data_root", "generate_gradcam", "find_image_path"]
missing = [v for v in required_vars if v not in dir()]
if missing:
    print("❌ Missing required variables from cell 11.5:", missing)
    print("   Run the Grad-CAM Visualization cell (11.5) first!")
else:
    print("✅ All prerequisites loaded")
    
    # Get ROI variant if using crops
    roi_var = globals().get("ROI_VARIANT") if any(globals().get(f"USE_{m}") for m in ["HYBRID", "YOLO", "MEDIAPIPE"]) else None
    
    # ============================================================================
    # ANALYZE EACH CONFUSION PAIR
    # ============================================================================
    for true_c, pred_c, description in CONFUSION_PAIRS:
        
        # Find samples with this confusion
        confused = preds_df[(preds_df["label"] == true_c) & (preds_df["pred"] == pred_c)]
        
        if len(confused) == 0:
            print(f"\n⬜ {description}: No examples found (model never makes this error)")
            continue
        
        print(f"\n{'='*70}")
        print(f"🔍 {description}")
        print(f"   Found {len(confused)} examples of this confusion")
        print(f"{'='*70}")
        
        # Sample up to MAX_SAMPLES
        samples = confused.head(MAX_SAMPLES)
        n_samples = len(samples)
        
        # Create figure
        fig, axes = plt.subplots(n_samples, 3, figsize=(12, n_samples * 3))
        if n_samples == 1:
            axes = axes.reshape(1, -1)
        
        for i, (_, row) in enumerate(samples.iterrows()):
            vis_path = row.get("vis_path", row.get("crop_path", row["path"]))
            img_path, _ = find_image_path(vis_path, data_root, DATASET_ROOT, roi_var)
            
            if img_path and img_path.exists():
                try:
                    # Load original image
                    img = Image.open(img_path).convert("RGB").resize((IMAGE_SIZE, IMAGE_SIZE))
                    axes[i, 0].imshow(img)
                    axes[i, 0].set_title(f"True: {CLASS_NAMES[true_c]}", fontsize=9)
                    axes[i, 0].axis("off")
                    
                    # Generate Grad-CAM
                    viz, heatmap = generate_gradcam(img_path)
                    
                    conf = row.get("confidence", 0)
                    axes[i, 1].imshow(viz)
                    axes[i, 1].set_title(f"Pred: {CLASS_NAMES[pred_c]} ({conf:.2f})", fontsize=9)
                    axes[i, 1].axis("off")
                    
                    # Heatmap only
                    axes[i, 2].imshow(heatmap, cmap="jet")
                    axes[i, 2].set_title("Attention Focus", fontsize=9)
                    axes[i, 2].axis("off")
                    
                except Exception as e:
                    print(f"   ⚠️ Error processing {vis_path}: {e}")
                    for j in range(3):
                        axes[i, j].text(0.5, 0.5, "Error", ha="center", va="center")
                        axes[i, j].axis("off")
            else:
                for j in range(3):
                    axes[i, j].text(0.5, 0.5, "Not found", ha="center", va="center")
                    axes[i, j].axis("off")
        
        plt.suptitle(f"Confusion Analysis: {description}\n({n_samples} examples)", fontsize=12, y=1.02)
        plt.tight_layout()
        
        # Save
        safe_name = description.replace(" ", "_").replace("→", "to").replace("/", "_")
        out_path = Path(OUT_ROOT) / "gradcam" / GRADCAM_TAG / "confusion_analysis" / f"{safe_name}.png"
        out_path.parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(out_path, dpi=150, bbox_inches="tight")
        plt.show()
        print(f"   💾 Saved to {out_path}")
        
        # ============================================================================
        # ANALYZE WHAT THE MODEL IS LOOKING AT
        # ============================================================================
        print(f"\n   📝 Analysis hints for '{description}':")
        if true_c in [1, 3] and pred_c in [2, 4]:  # Texting → Phone
            print("      - Look for: Is the attention on the phone, or on the ear/cheek area?")
            print("      - Texting = phone in hand, looking down. Phone = near ear/cheek.")
        elif true_c in [6] and pred_c in [7]:  # Drinking → Reaching
            print("      - Look for: Is the attention on the cup/bottle, or just the arm motion?")
            print("      - Both involve raised arm. Cup shape is the key differentiator.")
        elif true_c == 0 and pred_c in [1, 3]:  # Safe → Texting
            print("      - Look for: What spurious cue triggered the 'texting' prediction?")
            print("      - Could be dashboard, shadows, or similar-looking hand position.")
        elif true_c == 9 and pred_c in [0]:  # Passenger → Safe
            print("      - Look for: Is the model ignoring the head orientation/gaze?")
            print("      - Passenger involves turned head; Safe is forward-facing.")
        else:
            print("      - Examine where attention is focused vs. where it should be.")
            print("      - Look for background cues vs. action-relevant regions.")

    # ============================================================================
    # SUMMARY
    # ============================================================================
    print("\n" + "="*70)
    print("📊 CONFUSION PAIR SUMMARY")
    print("="*70)
    print(f"\nAnalyzed {len(CONFUSION_PAIRS)} confusion pairs")
    print(f"Visualizations saved to: {Path(OUT_ROOT) / 'gradcam' / GRADCAM_TAG / 'confusion_analysis'}")
    print("\nTo add more pairs, edit CONFUSION_PAIRS at the top of this cell.")


## 🔬 11.5a Grad-CAM Comparison: Full Image vs Hybrid Crops (Optional)

If you've trained both a **full-image model** and a **hybrid crop model**, run this cell to generate side-by-side comparisons showing how cropping changes the model's attention.

**Thesis insight:** This demonstrates WHY ROI cropping helps — the full-image model may attend to irrelevant regions while the crop model focuses on meaningful features.


In [ ]:
# 🔬 Grad-CAM Comparison: Full Image vs Hybrid Crops
# Requires: trained models for BOTH full images AND hybrid crops

import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision import transforms

# ===== CONFIGURATION =====
# Configure your models for comparison
# You need predictions from BOTH models. Run cell 11.3 separately for each.

# Model architecture (must be the same for both)
MODEL_NAME = "efficientnet_b0"            # Must be same architecture for both

# Split to analyze
SPLIT = "test"  # or "val"

# === FULL IMAGE MODEL ===
# The tag used for your full-image model training run
FULL_IMAGE_TAG = "effb0_full_images"      # <-- Change to your full-image model run tag
# Prediction file name (usually: <RUN_TAG>_<SPLIT>.csv)
FULL_PREDS_FILENAME = f"{FULL_IMAGE_TAG}_{SPLIT}.csv"  # e.g., effb0_full_images_test.csv

# === HYBRID CROP MODEL ===
# The tag used for your hybrid model training run
HYBRID_TAG = "effb0_InsightFace_MP_face_hands_V3"  # <-- Change to your hybrid model run tag
HYBRID_VARIANT = "face_hands"             # The ROI variant used
# Prediction file name
HYBRID_PREDS_FILENAME = f"{HYBRID_TAG}_{SPLIT}.csv"  # e.g., effb0_InsightFace_MP_face_hands_V3_test.csv

N_COMPARISON_SAMPLES = 6
IMAGE_SIZE = 224

# Print expected file locations
print("📁 Expected prediction files:")
print(f"   Full:   {Path(OUT_ROOT) / 'preds' / SPLIT / FULL_PREDS_FILENAME}")
print(f"   Hybrid: {Path(OUT_ROOT) / 'preds' / SPLIT / HYBRID_PREDS_FILENAME}")
print("   (If these don't exist, run cell 11.3 for each model first)")

# ===== HELPER FUNCTIONS =====
def load_model_and_cam(tag, model_name):
    """Load a model and create GradCAM for it."""
    from ddriver.models.registry import build_model, register_timm_backbone
    
    # Register timm backbone if needed
    try:
        register_timm_backbone(model_name)
    except (ImportError, ValueError):
        pass  # Already registered or custom model
    
    run_base = Path(CKPT_ROOT) / "runs" / tag
    all_runs = sorted(run_base.glob("*/"))
    if not all_runs:
        return None, None, None
    latest_run = all_runs[-1]
    ckpt_path = latest_run / "best.pt"
    if not ckpt_path.exists():
        ckpt_path = latest_run / "last.pt"
    
    ckpt = torch.load(ckpt_path, map_location="cpu")
    num_classes = ckpt["model_state_dict"].get("classifier.weight", torch.zeros(10, 1)).shape[0]
    
    model = build_model(model_name, num_classes=num_classes, pretrained=False)
    model.load_state_dict(ckpt["model_state_dict"])
    model.eval()
    
    # Get target layers - handle both wrapped and unwrapped models
    base = getattr(model, "backbone", model)
    if "efficientnet" in model_name.lower():
        if hasattr(base, "conv_head"):
            target_layers = [base.conv_head]
        elif hasattr(base, "blocks"):
            target_layers = [base.blocks[-1]]
        else:
            target_layers = [base.features[-1]]
    elif "resnet" in model_name.lower():
        target_layers = [base.layer4[-1]] if hasattr(base, "layer4") else [list(base.children())[-2]]
    elif "convnext" in model_name.lower():
        if hasattr(base, "stages"):
            target_layers = [base.stages[-1]]
        else:
            target_layers = [base.features[-1]]
    else:
        # Fallback
        for attr in ["features", "blocks", "stages"]:
            if hasattr(base, attr):
                target_layers = [getattr(base, attr)[-1]]
                break
        else:
            target_layers = [list(base.children())[-2]]
    
    cam = GradCAM(model=model, target_layers=target_layers)
    return model, cam, ckpt_path

# Load both models
print("Loading models...")
full_model, full_cam, full_ckpt = load_model_and_cam(FULL_IMAGE_TAG, MODEL_NAME)
hybrid_model, hybrid_cam, hybrid_ckpt = load_model_and_cam(HYBRID_TAG, MODEL_NAME)

if full_model is None:
    print(f"⚠️ Full image model not found: {FULL_IMAGE_TAG}")
    print("   Train a full-image model first, or update FULL_IMAGE_TAG")
if hybrid_model is None:
    print(f"⚠️ Hybrid model not found: {HYBRID_TAG}")
    print("   Train a hybrid model first, or update HYBRID_TAG")

if full_model is None or hybrid_model is None:
    raise RuntimeError("Both models required for comparison. Check tags above.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
full_model = full_model.to(device)
hybrid_model = hybrid_model.to(device)
print(f"✅ Loaded both models")

# Transform
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load predictions to find common samples
full_preds_path = Path(OUT_ROOT) / "preds" / SPLIT / FULL_PREDS_FILENAME
hybrid_preds_path = Path(OUT_ROOT) / "preds" / SPLIT / HYBRID_PREDS_FILENAME

if not full_preds_path.exists():
    raise FileNotFoundError(
        f"❌ Full-image predictions not found: {full_preds_path}\n"
        f"   To generate: Run cell 11.2 with FULL_IMAGE_TAG checkpoint, then run cell 11.3"
    )
if not hybrid_preds_path.exists():
    raise FileNotFoundError(
        f"❌ Hybrid predictions not found: {hybrid_preds_path}\n"
        f"   To generate: Run cell 11.2 with HYBRID_TAG checkpoint, then run cell 11.3"
    )

full_preds = pd.read_csv(full_preds_path)
hybrid_preds = pd.read_csv(hybrid_preds_path)
print(f"📊 Loaded {len(full_preds)} full-image predictions, {len(hybrid_preds)} hybrid predictions")

# Convert pred_class_id (c0-c9) to integers for display
def class_to_int(class_id):
    if isinstance(class_id, str) and class_id.startswith("c"):
        return int(class_id[1:])
    return int(class_id) if pd.notna(class_id) else -1

full_preds["pred"] = full_preds["pred_class_id"].apply(class_to_int)
hybrid_preds["pred"] = hybrid_preds["pred_class_id"].apply(class_to_int)

# Full images path and hybrid crops path
full_data_root = Path(DATASET_ROOT)
hybrid_data_root = Path(os.environ.get("HYBRID_ROOT_LOCAL", Path(OUT_ROOT) / "hybrid"))

# Load hybrid manifest once (not in loop)
hybrid_manifest_path = hybrid_data_root / f"manifest_{HYBRID_VARIANT}.csv"
if not hybrid_manifest_path.exists():
    raise FileNotFoundError(f"❌ Hybrid manifest not found: {hybrid_manifest_path}")
hybrid_manifest = pd.read_csv(hybrid_manifest_path)
print(f"📋 Loaded hybrid manifest with {len(hybrid_manifest)} entries")

# Sample images
samples = full_preds.sample(min(N_COMPARISON_SAMPLES, len(full_preds)), random_state=42)

# Generate comparison
fig, axes = plt.subplots(len(samples), 4, figsize=(16, len(samples) * 3.5))

CLASS_NAMES = {
    0: "Safe", 1: "Text-R", 2: "Talk-R", 3: "Text-L", 4: "Talk-L",
    5: "Radio", 6: "Drink", 7: "Reach", 8: "Hair", 9: "Passenger"
}

for i, (_, row) in enumerate(samples.iterrows()):
    # Find corresponding hybrid path
    full_path = full_data_root / row["path"]
    
    # Match in hybrid manifest by finding same original image
    # The hybrid manifest has "original_path" column that links to original images
    orig_filename = Path(row["path"]).name
    match = hybrid_manifest[hybrid_manifest["original_path"].str.contains(orig_filename, na=False)]
    
    if len(match) == 0:
        continue
    hybrid_path = hybrid_data_root / match.iloc[0]["path"]
    
    # Load and process full image
    if full_path.exists():
        full_img = Image.open(full_path).convert("RGB")
        full_img_resized = full_img.resize((IMAGE_SIZE, IMAGE_SIZE))
        full_np = np.array(full_img_resized) / 255.0
        full_tensor = transform(full_img).unsqueeze(0).to(device)
        
        full_cam_result = full_cam(input_tensor=full_tensor, targets=None)[0]
        full_viz = show_cam_on_image(full_np.astype(np.float32), full_cam_result, use_rgb=True)
        
        axes[i, 0].imshow(full_img_resized)
        axes[i, 0].set_title(f"Full Image\nTrue: {CLASS_NAMES.get(int(row['label']), row['label'])}", fontsize=9)
        axes[i, 0].axis("off")
        
        axes[i, 1].imshow(full_viz)
        axes[i, 1].set_title(f"Full Grad-CAM\nPred: {CLASS_NAMES.get(int(row['pred']), row['pred'])}", fontsize=9)
        axes[i, 1].axis("off")
    
    # Load and process hybrid crop
    if hybrid_path.exists():
        hybrid_img = Image.open(hybrid_path).convert("RGB")
        hybrid_img_resized = hybrid_img.resize((IMAGE_SIZE, IMAGE_SIZE))
        hybrid_np = np.array(hybrid_img_resized) / 255.0
        hybrid_tensor = transform(hybrid_img).unsqueeze(0).to(device)
        
        hybrid_cam_result = hybrid_cam(input_tensor=hybrid_tensor, targets=None)[0]
        hybrid_viz = show_cam_on_image(hybrid_np.astype(np.float32), hybrid_cam_result, use_rgb=True)
        
        axes[i, 2].imshow(hybrid_img_resized)
        axes[i, 2].set_title(f"Hybrid Crop", fontsize=9)
        axes[i, 2].axis("off")
        
        axes[i, 3].imshow(hybrid_viz)
        hybrid_row = hybrid_preds[hybrid_preds["path"].str.contains(Path(hybrid_path).name, na=False)]
        pred_label = hybrid_row.iloc[0]["pred"] if len(hybrid_row) > 0 else "?"
        axes[i, 3].set_title(f"Hybrid Grad-CAM\nPred: {CLASS_NAMES.get(int(pred_label), pred_label)}", fontsize=9)
        axes[i, 3].axis("off")

plt.suptitle(f"Grad-CAM Comparison: Full Image vs Hybrid Crop ({MODEL_NAME})", fontsize=14, y=1.02)
plt.tight_layout()

# Save
comparison_path = Path(OUT_ROOT) / "gradcam" / "comparison_full_vs_hybrid.png"
comparison_path.parent.mkdir(parents=True, exist_ok=True)
plt.savefig(comparison_path, dpi=150, bbox_inches="tight")
plt.show()
print(f"\n💾 Saved comparison to: {comparison_path}")
print("\n📝 Thesis talking point:")
print("   'The full-image model attends to [background/seat/etc] while the")
print("    hybrid-crop model focuses on [hand position/facial features/etc],")
print("    demonstrating the value of ROI-based preprocessing.'")
